<a href="https://colab.research.google.com/github/SylvanusAdonu/hello-world/blob/master/SPX_Greeks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
import warnings
warnings.filterwarnings('ignore')

# Import Plotly libraries
import plotly.graph_objects as go
from plotly.subplots import make_subplots

class GammaExposureAnalyzer:
    def __init__(self, symbol="QQQ"):  # Changed default from "^SPX" to "QQQ"
        self.symbol = symbol
        self.ticker = yf.Ticker(symbol)
        self.underlying_price = None
        self.expiration_dates = None
        self.risk_free_rate = 0.02  # Risk-free rate for calculations

    def get_current_price(self):
        """Get current underlying price"""
        hist = pd.DataFrame()
        try:
            hist = self.ticker.history(period='1d', interval='1m')
            if hist.empty:
                print("No 1-minute data, trying daily...")
                hist = self.ticker.history(period='1d')
        except Exception as e:
            print(f"Error fetching 1-minute data: {e}, trying daily...")
            hist = self.ticker.history(period='1d')

        if not hist.empty:
            self.underlying_price = hist['Close'].iloc[-1]
        else:
            self.underlying_price = None
            print(f"Warning: Could not retrieve price for {self.symbol}.")

        return self.underlying_price

    def get_options_chain(self, expiration_date):
        """Get options chain for specific expiration"""
        try:
            chain = self.ticker.option_chain(expiration_date)
            return chain.calls, chain.puts
        except Exception as e:
            print(f"Error fetching options chain: {e}")
            return None, None

    # ----------------------------------------------------------------
    # GREEK CALCULATION HELPER FUNCTIONS
    # ----------------------------------------------------------------
    def _calculate_d1_d2(self, S, K, T, sigma):
        """Helper to calculate d1 and d2 for Black-Scholes"""
        if T <= 0 or sigma <= 0:
            return None, None

        d1 = (np.log(S / K) + (self.risk_free_rate + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        return d1, d2

    def calculate_delta(self, option_type, S, K, T, sigma):
        """Calculate delta using Black-Scholes"""
        d1, _ = self._calculate_d1_d2(S, K, T, sigma)
        if d1 is None:
            return 0

        if option_type.upper() == 'CALL':
            from scipy.stats import norm
            return norm.cdf(d1)
        elif option_type.upper() == 'PUT':
            from scipy.stats import norm
            return norm.cdf(d1) - 1
        return 0

    def calculate_gamma(self, S, K, T, sigma):
        """Calculate gamma using Black-Scholes"""
        d1, _ = self._calculate_d1_d2(S, K, T, sigma)
        if d1 is None:
            return 0

        from scipy.stats import norm
        return norm.pdf(d1) / (S * sigma * np.sqrt(T))

    def calculate_vega(self, S, K, T, sigma):
        """Calculate vega using Black-Scholes"""
        d1, _ = self._calculate_d1_d2(S, K, T, sigma)
        if d1 is None:
            return 0

        from scipy.stats import norm
        return S * norm.pdf(d1) * np.sqrt(T) * 0.01  # 0.01 for 1% change in volatility

    def calculate_vanna(self, S, K, T, sigma):
        """Calculate vanna (dDelta/dVolatility)"""
        d1, d2 = self._calculate_d1_d2(S, K, T, sigma)
        if d1 is None:
            return 0

        from scipy.stats import norm
        return -norm.pdf(d1) * d2 / sigma

    # ----------------------------------------------------------------
    # NEW GREEK EXPOSURE CALCULATIONS
    # ----------------------------------------------------------------
    def calculate_delta_adjusted_gamma(self, expiration_date):
        """
        Calculate Delta-Adjusted Gamma Exposure
        DAG = Gamma * Delta * OpenInterest * 100 * Spot^2
        """
        calls, puts = self.get_options_chain(expiration_date)
        if calls is None or puts is None:
            return None

        # Get time to expiration
        today = datetime.now()
        if isinstance(expiration_date, str):
            expiry_date = datetime.strptime(expiration_date, '%Y-%m-%d').date()
        else:
            expiry_date = expiration_date.date()

        days_to_expiry = (expiry_date - today.date()).days + 1
        if days_to_expiry <= 0:
            days_to_expiry = 0.001

        T = days_to_expiry / 365.0
        S = self.underlying_price

        # Prepare dataframes
        calls['type'] = 'CALL'
        puts['type'] = 'PUT'
        all_options = pd.concat([calls, puts])

        # Calculate delta and gamma for each option
        dag_list = []
        for _, row in all_options.iterrows():
            iv = row.get('impliedVolatility', 0.3)
            K = row['strike']

            delta = self.calculate_delta(row['type'], S, K, T, iv)
            gamma = self.calculate_gamma(S, K, T, iv)

            # Delta-Adjusted Gamma Exposure
            dag = gamma * delta * row['openInterest'] * 100 * (S ** 2)
            dag_list.append(dag)

        all_options['dag'] = dag_list

        # Separate and merge
        calls_dag = all_options[all_options['type'] == 'CALL'][['strike', 'dag']].rename(
            columns={'dag': 'call_dag'})
        puts_dag = all_options[all_options['type'] == 'PUT'][['strike', 'dag']].rename(
            columns={'dag': 'put_dag'})

        merged = pd.merge(calls_dag, puts_dag, on='strike', how='outer').fillna(0)
        merged['net_dag'] = merged['call_dag'] - merged['put_dag']
        merged['abs_dag'] = abs(merged['call_dag']) + abs(merged['put_dag'])

        return merged

    def calculate_vex(self, expiration_date):
        """
        Calculate Vega Exposure (VEX)
        VEX = Vega * OpenInterest * 100 * Spot
        """
        calls, puts = self.get_options_chain(expiration_date)
        if calls is None or puts is None:
            return None

        # Get time to expiration
        today = datetime.now()
        if isinstance(expiration_date, str):
            expiry_date = datetime.strptime(expiration_date, '%Y-%m-%d').date()
        else:
            expiry_date = expiration_date.date()

        days_to_expiry = (expiry_date - today.date()).days + 1
        if days_to_expiry <= 0:
            days_to_expiry = 0.001

        T = days_to_expiry / 365.0
        S = self.underlying_price

        # Prepare dataframes
        calls['type'] = 'CALL'
        puts['type'] = 'PUT'
        all_options = pd.concat([calls, puts])

        # Calculate vega for each option
        vex_list = []
        for _, row in all_options.iterrows():
            iv = row.get('impliedVolatility', 0.3)
            K = row['strike']

            vega = self.calculate_vega(S, K, T, iv)
            vex = vega * row['openInterest'] * 100 * S
            vex_list.append(vex)

        all_options['vex'] = vex_list

        # Separate and merge
        calls_vex = all_options[all_options['type'] == 'CALL'][['strike', 'vex']].rename(
            columns={'vex': 'call_vex'})
        puts_vex = all_options[all_options['type'] == 'PUT'][['strike', 'vex']].rename(
            columns={'vex': 'put_vex'})

        merged = pd.merge(calls_vex, puts_vex, on='strike', how='outer').fillna(0)
        merged['net_vex'] = merged['call_vex'] + merged['put_vex']  # Vega is positive for both
        merged['abs_vex'] = merged['call_vex'] + merged['put_vex']

        return merged

    def calculate_vanna_exposure(self, expiration_date):
        """
        Calculate Vanna Exposure
        Vanna Exposure = Vanna * OpenInterest * 100 * Spot^2
        """
        calls, puts = self.get_options_chain(expiration_date)
        if calls is None or puts is None:
            return None

        # Get time to expiration
        today = datetime.now()
        if isinstance(expiration_date, str):
            expiry_date = datetime.strptime(expiration_date, '%Y-%m-%d').date()
        else:
            expiry_date = expiration_date.date()

        days_to_expiry = (expiry_date - today.date()).days + 1
        if days_to_expiry <= 0:
            days_to_expiry = 0.001

        T = days_to_expiry / 365.0
        S = self.underlying_price

        # Prepare dataframes
        calls['type'] = 'CALL'
        puts['type'] = 'PUT'
        all_options = pd.concat([calls, puts])

        # Calculate vanna for each option
        vanna_exp_list = []
        for _, row in all_options.iterrows():
            iv = row.get('impliedVolatility', 0.3)
            K = row['strike']

            vanna = self.calculate_vanna(S, K, T, iv)
            vanna_exp = vanna * row['openInterest'] * 100 * (S ** 2)
            vanna_exp_list.append(vanna_exp)

        all_options['vanna_exp'] = vanna_exp_list

        # Separate and merge
        calls_vanna = all_options[all_options['type'] == 'CALL'][['strike', 'vanna_exp']].rename(
            columns={'vanna_exp': 'call_vanna_exp'})
        puts_vanna = all_options[all_options['type'] == 'PUT'][['strike', 'vanna_exp']].rename(
            columns={'vanna_exp': 'put_vanna_exp'})

        merged = pd.merge(calls_vanna, puts_vanna, on='strike', how='outer').fillna(0)
        merged['net_vanna_exp'] = merged['call_vanna_exp'] + merged['put_vanna_exp']
        merged['abs_vanna_exp'] = abs(merged['call_vanna_exp']) + abs(merged['put_vanna_exp'])

        return merged

    # ----------------------------------------------------------------
    # EXISTING FUNCTIONS (modified for QQQ)
    # ----------------------------------------------------------------
    def calculate_gex(self, expiration_date):
        """Calculate Gamma Exposure for a specific expiration - Updated to use Black-Scholes"""
        calls, puts = self.get_options_chain(expiration_date)

        if calls is None or puts is None:
            return None, None

        # Get time to expiration
        today = datetime.now()
        if isinstance(expiration_date, str):
            expiry_date = datetime.strptime(expiration_date, '%Y-%m-%d').date()
        else:
            expiry_date = expiration_date.date()

        days_to_expiry = (expiry_date - today.date()).days + 1
        if days_to_expiry <= 0:
            days_to_expiry = 0.001

        T = days_to_expiry / 365.0
        S = self.underlying_price

        # Prepare dataframes
        calls['type'] = 'CALL'
        puts['type'] = 'PUT'
        all_options = pd.concat([calls, puts])

        # Calculate gamma for each option using Black-Scholes
        gamma_list = []
        for _, row in all_options.iterrows():
            iv = row.get('impliedVolatility', 0.3)
            K = row['strike']
            gamma = self.calculate_gamma(S, K, T, iv)
            gamma_list.append(gamma)

        all_options['gamma'] = gamma_list
        all_options['gex_oi'] = all_options['gamma'] * all_options['openInterest'] * 100 * (S ** 2)
        all_options['gex_volume'] = all_options['gamma'] * all_options['volume'] * 100 * (S ** 2)

        # Separate calls and puts
        calls_df = all_options[all_options['type'] == 'CALL'].copy()
        puts_df = all_options[all_options['type'] == 'PUT'].copy()

        # Merge based on strike
        merged = pd.merge(
            calls_df[['strike', 'gex_oi', 'gex_volume']].rename(
                columns={'gex_oi': 'call_gex_oi', 'gex_volume': 'call_gex_volume'}
            ),
            puts_df[['strike', 'gex_oi', 'gex_volume']].rename(
                columns={'gex_oi': 'put_gex_oi', 'gex_volume': 'put_gex_volume'}
            ),
            on='strike',
            how='outer'
        ).fillna(0)

        merged['net_gex_oi'] = merged['call_gex_oi'] - merged['put_gex_oi']
        merged['abs_gex_oi'] = merged['call_gex_oi'] + merged['put_gex_oi']
        merged['net_gex_volume'] = merged['call_gex_volume'] - merged['put_gex_volume']
        merged['abs_gex_volume'] = merged['call_gex_volume'] + merged['put_gex_volume']

        return merged, all_options

    def calculate_dex(self, expiration_date):
        """Calculate Delta Exposure for a specific expiration - Updated to use Black-Scholes"""
        calls, puts = self.get_options_chain(expiration_date)

        if calls is None or puts is None:
            return None

        # Get time to expiration
        today = datetime.now()
        if isinstance(expiration_date, str):
            expiry_date = datetime.strptime(expiration_date, '%Y-%m-%d').date()
        else:
            expiry_date = expiration_date.date()

        days_to_expiry = (expiry_date - today.date()).days + 1
        if days_to_expiry <= 0:
            days_to_expiry = 0.001

        T = days_to_expiry / 365.0
        S = self.underlying_price

        # Prepare dataframes
        calls['type'] = 'CALL'
        puts['type'] = 'PUT'
        all_options = pd.concat([calls, puts])

        # Calculate delta using Black-Scholes
        delta_list = []
        for _, row in all_options.iterrows():
            iv = row.get('impliedVolatility', 0.3)
            K = row['strike']
            delta = self.calculate_delta(row['type'], S, K, T, iv)
            delta_list.append(delta)

        all_options['delta'] = delta_list
        all_options['dex_oi'] = all_options['delta'] * all_options['openInterest'] * 100 * S
        all_options['dex_volume'] = all_options['delta'] * all_options['volume'] * 100 * S

        # Separate and aggregate
        calls_df = all_options[all_options['type'] == 'CALL'][['strike', 'dex_oi', 'dex_volume']]
        puts_df = all_options[all_options['type'] == 'PUT'][['strike', 'dex_oi', 'dex_volume']]

        merged = pd.merge(
            calls_df.rename(columns={'dex_oi': 'call_dex_oi', 'dex_volume': 'call_dex_volume'}),
            puts_df.rename(columns={'dex_oi': 'put_dex_oi', 'dex_volume': 'put_dex_volume'}),
            on='strike',
            how='outer'
        ).fillna(0)

        merged['net_dex_oi'] = merged['call_dex_oi'] + merged['put_dex_oi']
        merged['abs_dex_oi'] = abs(merged['call_dex_oi']) + abs(merged['put_dex_oi'])
        merged['net_dex_volume'] = merged['call_dex_volume'] + merged['put_dex_volume']
        merged['abs_dex_volume'] = abs(merged['call_dex_volume']) + abs(merged['put_dex_volume'])

        return merged

    def plot_all_metrics(self, expiration_date):
        """Create comprehensive plots of all metrics including new Greeks"""
        # Get current price
        self.get_current_price()

        # Calculate all exposures
        dex_df = self.calculate_dex(expiration_date)
        gex_df_tuple = self.calculate_gex(expiration_date)
        gex_df = gex_df_tuple[0] if gex_df_tuple else None # Extract gex_df from tuple
        all_options = gex_df_tuple[1] if gex_df_tuple else None # Extract all_options from tuple

        dag_df = self.calculate_delta_adjusted_gamma(expiration_date)
        vex_df = self.calculate_vex(expiration_date)
        vanna_df = self.calculate_vanna_exposure(expiration_date)

        # Check if any required DataFrame is None or empty
        if any(df is None or (isinstance(df, pd.DataFrame) and df.empty) for df in [dex_df, gex_df, dag_df, vex_df, vanna_df]):
            print("Failed to fetch data for one or more metrics or received empty data.")
            return

        # Merge all data
        merged_df = pd.merge(dex_df, gex_df, on='strike', how='outer')
        merged_df = pd.merge(merged_df, dag_df, on='strike', how='outer')
        merged_df = pd.merge(merged_df, vex_df, on='strike', how='outer')
        merged_df = pd.merge(merged_df, vanna_df, on='strike', how='outer').fillna(0)

        # Filter for relevant strikes
        if self.underlying_price:
            price_range = 0.20
            min_strike = self.underlying_price * (1 - price_range)
            max_strike = self.underlying_price * (1 + price_range)
            filtered_df = merged_df[(merged_df['strike'] >= min_strike) & (merged_df['strike'] <= max_strike)]
        else:
            filtered_df = merged_df

        # Create subplots - 4 rows, 3 columns
        fig = make_subplots(
            rows=4, cols=3,
            subplot_titles=(
                'DEX (Delta Exposure) - OI',
                'Net GEX (Gamma Exposure) - OI',
                'Delta-Adjusted Gamma - OI',
                'Absolute GEX - OI',
                'Vega Exposure (VEX) - OI',
                'Vanna Exposure - OI',
                'DEX Comparison: OI vs Volume',
                'GEX Comparison: OI vs Volume',
                'VEX Comparison: OI',
                'Gamma Profile (Net GEX)',
                'Vanna Profile',
                'All Exposures Overview'
            ),
            vertical_spacing=0.08,
            horizontal_spacing=0.1
        )

        # Helper function to add spot price line
        def add_spot_line(row, col):
            if self.underlying_price:
                fig.add_vline(x=self.underlying_price, line_dash="dash",
                            line_color="red", line_width=1, row=row, col=col)

        # 1. DEX (Delta Exposure) - OI
        fig.add_trace(
            go.Bar(x=filtered_df['strike'], y=filtered_df['net_dex_oi'],
                  name='Net DEX OI', marker_color='blue'),
            row=1, col=1
        )
        fig.add_hline(y=0, line_dash="dash", line_color="black", row=1, col=1)
        add_spot_line(1, 1)

        # 2. Net GEX - OI
        fig.add_trace(
            go.Bar(x=filtered_df['strike'], y=filtered_df['net_gex_oi'],
                  name='Net GEX OI', marker_color='green'),
            row=1, col=2
        )
        fig.add_hline(y=0, line_dash="dash", line_color="black", row=1, col=2)
        add_spot_line(1, 2)

        # 3. Delta-Adjusted Gamma - OI
        fig.add_trace(
            go.Bar(x=filtered_df['strike'], y=filtered_df['net_dag'],
                  name='Net DAG', marker_color='purple'),
            row=1, col=3
        )
        fig.add_hline(y=0, line_dash="dash", line_color="black", row=1, col=3)
        add_spot_line(1, 3)

        # 4. Absolute GEX - OI
        fig.add_trace(
            go.Bar(x=filtered_df['strike'], y=filtered_df['abs_gex_oi'],
                  name='Abs GEX OI', marker_color='orange'),
            row=2, col=1
        )
        add_spot_line(2, 1)

        # 5. Vega Exposure - OI
        fig.add_trace(
            go.Bar(x=filtered_df['strike'], y=filtered_df['net_vex'],
                  name='Net VEX', marker_color='cyan'),
            row=2, col=2
        )
        fig.add_hline(y=0, line_dash="dash", line_color="black", row=2, col=2)
        add_spot_line(2, 2)

        # 6. Vanna Exposure - OI
        fig.add_trace(
            go.Bar(x=filtered_df['strike'], y=filtered_df['net_vanna_exp'],
                  name='Net Vanna', marker_color='magenta'),
            row=2, col=3
        )
        fig.add_hline(y=0, line_dash="dash", line_color="black", row=2, col=3)
        add_spot_line(2, 3)

        # 7. DEX Comparison
        fig.add_trace(
            go.Bar(x=filtered_df['strike'], y=filtered_df['net_dex_oi'],
                  name='DEX OI', marker_color='blue', opacity=0.7),
            row=3, col=1
        )
        fig.add_trace(
            go.Bar(x=filtered_df['strike'], y=filtered_df['abs_dex_oi'],
                  name='Abs DEX OI', marker_color='lightblue', opacity=0.7),
            row=3, col=1
        )
        fig.add_hline(y=0, line_dash="dash", line_color="black", row=3, col=1)
        add_spot_line(3, 1)

        # 8. GEX Comparison
        fig.add_trace(
            go.Bar(x=filtered_df['strike'], y=filtered_df['net_gex_oi'],
                  name='Net GEX OI', marker_color='green', opacity=0.7),
            row=3, col=2
        )
        fig.add_trace(
            go.Bar(x=filtered_df['strike'], y=filtered_df['abs_gex_oi'],
                  name='Abs GEX OI', marker_color='lightgreen', opacity=0.7),
            row=3, col=2
        )
        fig.add_hline(y=0, line_dash="dash", line_color="black", row=3, col=2)
        add_spot_line(3, 2)

        # 9. VEX Comparison
        fig.add_trace(
            go.Scatter(x=filtered_df['strike'], y=filtered_df['net_vex'],
                      mode='lines+markers', name='Net VEX',
                      line=dict(color='cyan', width=2)),
            row=3, col=3
        )
        fig.add_hline(y=0, line_dash="dash", line_color="black", row=3, col=3)
        add_spot_line(3, 3)

        # 10. Gamma Profile
        fig.add_trace(
            go.Scatter(x=filtered_df['strike'], y=filtered_df['net_gex_oi'],
                      mode='lines', name='Net GEX OI',
                      line=dict(color='green', width=2)),
            row=4, col=1
        )
        fig.add_trace(
            go.Scatter(x=filtered_df['strike'], y=filtered_df['abs_gex_oi'],
                      mode='lines', name='Abs GEX OI',
                      line=dict(color='orange', width=2, dash='dash')),
            row=4, col=1
        )
        fig.add_hline(y=0, line_dash="dash", line_color="black", row=4, col=1)
        add_spot_line(4, 1)

        # 11. Vanna Profile
        fig.add_trace(
            go.Scatter(x=filtered_df['strike'], y=filtered_df['net_vanna_exp'],
                      mode='lines', name='Net Vanna',
                      line=dict(color='magenta', width=2)),
            row=4, col=2
        )
        fig.add_trace(
            go.Scatter(x=filtered_df['strike'], y=filtered_df['abs_vanna_exp'],
                      mode='lines', name='Abs Vanna',
                      line=dict(color='pink', width=2, dash='dash')),
            row=4, col=2
        )
        fig.add_hline(y=0, line_dash="dash", line_color="black", row=4, col=2)
        add_spot_line(4, 2)

        # 12. All Exposures Overview (normalized)
        from sklearn.preprocessing import MinMaxScaler
        scaler = MinMaxScaler()

        overview_cols = ['net_dex_oi', 'net_gex_oi', 'net_dag', 'net_vex', 'net_vanna_exp']
        overview_data = filtered_df[overview_cols].copy()

        # Normalize each column for comparison
        for col in overview_cols:
            if not overview_data[col].empty and overview_data[col].max() != overview_data[col].min():
                overview_data[col] = scaler.fit_transform(overview_data[[col]])
            else:
                overview_data[col] = 0

        colors = ['blue', 'green', 'purple', 'cyan', 'magenta']
        for i, col in enumerate(overview_cols):
            fig.add_trace(
                go.Scatter(x=filtered_df['strike'], y=overview_data[col],
                          mode='lines', name=col.replace('net_', '').replace('_oi', '').upper(),
                          line=dict(color=colors[i], width=2)),
                row=4, col=3
            )
        add_spot_line(4, 3)

        # Update layout
        fig.update_layout(
            title_text=f'QQQ Options Analysis - Expiry: {expiration_date}<br>Current Price: ${self.underlying_price:.2f}' if self.underlying_price else f'QQQ Options Analysis - Expiry: {expiration_date}',
            height=1400, width=1500,
            showlegend=True,
            hovermode='x unified',
            legend=dict(orientation="h", yanchor="bottom", y=-0.1, xanchor="center", x=0.5)
        )

        # Update axes
        for i in range(1, 5):
            for j in range(1, 4):
                fig.update_xaxes(title_text='Strike Price', row=i, col=j)
                fig.update_yaxes(title_text='Exposure', row=i, col=j)

        fig.show()

        # Print summary statistics
        print("\n" + "="*80)
        print("QQQ OPTIONS EXPOSURE SUMMARY")
        print("="*80)
        print(f"Underlying Price: ${self.underlying_price:.2f}" if self.underlying_price else "Underlying Price: N/A")
        print(f"Expiration: {expiration_date}")
        if not filtered_df.empty:
            print(f"\n--- Open Interest Based Exposures ---")
            print(f"Total Net DEX:      ${filtered_df['net_dex_oi'].sum()/1e9:.3f}B")
            print(f"Total Net GEX:      ${filtered_df['net_gex_oi'].sum()/1e9:.3f}B")
            print(f"Total Abs GEX:      ${filtered_df['abs_gex_oi'].sum()/1e9:.3f}B")
            print(f"Total Delta-Adj Gamma: ${filtered_df['net_dag'].sum()/1e9:.3f}B")
            print(f"Total Net VEX:      ${filtered_df['net_vex'].sum()/1e9:.3f}B")
            print(f"Total Net Vanna:    ${filtered_df['net_vanna_exp'].sum()/1e9:.3f}B")

            # Find key levels
            print(f"\n--- Key Strike Levels ---")
            if 'net_gex_oi' in filtered_df.columns and not filtered_df['net_gex_oi'].empty and not filtered_df['net_gex_oi'].isnull().all():
                max_gex_strike = filtered_df.loc[filtered_df['net_gex_oi'].abs().idxmax(), 'strike']
                print(f"Strike with Max |GEX|: ${max_gex_strike:.2f}")
            else:
                print("Could not determine Max |GEX| strike.")

            if 'net_vanna_exp' in filtered_df.columns and not filtered_df['net_vanna_exp'].empty and not filtered_df['net_vanna_exp'].isnull().all():
                max_vanna_strike = filtered_df.loc[filtered_df['net_vanna_exp'].abs().idxmax(), 'strike']
                print(f"Strike with Max |Vanna|: ${max_vanna_strike:.2f}")
            else:
                print("Could not determine Max |Vanna| strike.")
        else:
            print("No filtered data available for summary statistics.")

        return filtered_df

    def plot_qqq_with_greeks_levels(self):  # Renamed from plot_spx_with_greeks_levels
        """Fetches QQQ data and plots with Greek levels"""
        # Fetch data
        qqq_data = pd.DataFrame()
        try:
            qqq_data = self.ticker.history(period='1d', interval='1m')
            if qqq_data.empty:
                print("No 1-minute data available. Using daily data.")
                qqq_data = self.ticker.history(period='5d')
        except Exception as e:
            print(f"Error fetching QQQ data: {e}")
            qqq_data = self.ticker.history(period='5d')

        # Get current price
        self.get_current_price()
        if self.underlying_price is None:
            print("Cannot plot Greek levels without current price.")
            return

        # Get nearest expiry
        expirations = self.ticker.options
        if not expirations:
            print("No options data available.")
            return

        nearest_expiry = expirations[0]
        print(f"Analyzing nearest expiration: {nearest_expiry}")

        # Calculate all exposures
        dex_df = self.calculate_dex(nearest_expiry)
        gex_df_tuple = self.calculate_gex(nearest_expiry)
        gex_df = gex_df_tuple[0] if gex_df_tuple else None # Correctly extract gex_df from tuple

        dag_df = self.calculate_delta_adjusted_gamma(nearest_expiry)
        vex_df = self.calculate_vex(nearest_expiry)
        vanna_df = self.calculate_vanna_exposure(nearest_expiry)

        # Check if any required DataFrame is None or empty
        if any(df is None or (isinstance(df, pd.DataFrame) and df.empty) for df in [dex_df, gex_df, dag_df, vex_df, vanna_df]):
            print("Failed to calculate one or more exposures or received empty data.")
            return

        # Merge and filter
        all_dfs = [dex_df, gex_df, dag_df, vex_df, vanna_df]
        merged_df = all_dfs[0]
        for df in all_dfs[1:]:
            merged_df = pd.merge(merged_df, df, on='strike', how='outer')
        merged_df = merged_df.fillna(0)

        price_range = 0.20
        min_strike = self.underlying_price * (1 - price_range)
        max_strike = self.underlying_price * (1 + price_range)
        filtered_df = merged_df[(merged_df['strike'] >= min_strike) &
                               (merged_df['strike'] <= max_strike)]

        # Find significant strikes
        sig_levels = {}
        metrics = [
            ('abs_dex_oi', 'Max DEX', 'blue'),
            ('abs_gex_oi', 'Max GEX', 'green'),
            ('abs_dag', 'Max DAG', 'purple'),
            ('abs_vex', 'Max VEX', 'cyan'),
            ('abs_vanna_exp', 'Max Vanna', 'magenta')
        ]

        for metric, label, color in metrics:
            if metric in filtered_df.columns and not filtered_df[metric].empty and not filtered_df[metric].isnull().all():
                idx = filtered_df[metric].idxmax()
                strike_val = filtered_df.loc[idx, 'strike']
                metric_val = filtered_df.loc[idx, metric]
                sig_levels[f"Abs {label} (${strike_val:.2f})"] = {
                    'strike': strike_val,
                    'value': metric_val,
                    'color': color
                }

        # Add net GEX OI (positive and negative peaks)
        if 'net_gex_oi' in filtered_df.columns and not filtered_df['net_gex_oi'].empty and not filtered_df['net_gex_oi'].isnull().all():
            positive_net_gex_oi = filtered_df[filtered_df['net_gex_oi'] > 0]
            if not positive_net_gex_oi.empty:
                max_pos_net_gex_oi_strike = positive_net_gex_oi.loc[positive_net_gex_oi['net_gex_oi'].idxmax()]['strike']
                sig_levels[f"Max Pos Net GEX OI (${max_pos_net_gex_oi_strike:.2f})"] = {
                    'strike': max_pos_net_gex_oi_strike,
                    'value': positive_net_gex_oi['net_gex_oi'].max(),
                    'color': 'orange'
                }

            negative_net_gex_oi = filtered_df[filtered_df['net_gex_oi'] < 0]
            if not negative_net_gex_oi.empty:
                min_neg_net_gex_oi_strike = negative_net_gex_oi.loc[negative_net_gex_oi['net_gex_oi'].idxmin()]['strike']
                sig_levels[f"Max Neg Net GEX OI (${min_neg_net_gex_oi_strike:.2f})"] = {
                    'strike': min_neg_net_gex_oi_strike,
                    'value': negative_net_gex_oi['net_gex_oi'].min(),
                    'color': 'red'
                }

        # Create plot
        fig = go.Figure()

        # Add candlestick if available
        if not qqq_data.empty and 'Open' in qqq_data.columns and len(qqq_data) > 10:
            fig.add_trace(go.Candlestick(
                x=qqq_data.index,
                open=qqq_data['Open'],
                high=qqq_data['High'],
                low=qqq_data['Low'],
                close=qqq_data['Close'],
                name='QQQ',
                increasing_line_color='green',
                decreasing_line_color='red'
            ))
        elif not qqq_data.empty and 'Close' in qqq_data.columns:
            # Line plot as fallback for less than 10 points or no Open/High/Low
            fig.add_trace(go.Scatter(
                x=qqq_data.index,
                y=qqq_data['Close'],
                mode='lines',
                name='QQQ',
                line=dict(color='blue', width=2)
            ))
        else:
            print("No sufficient QQQ data to plot candlestick or line chart.")

        # Add horizontal lines for Greek levels
        # Sort levels by strike price for better annotation placement
        sorted_levels = sorted(sig_levels.items(), key=lambda item: item[1]['strike'])

        # Define annotation positions to try to avoid overlaps
        positions = ["top right", "bottom right", "top left", "bottom left", "right", "left"]
        pos_idx = 0

        for label, info in sorted_levels:
            fig.add_hline(
                y=info['strike'],
                line_dash="dash",
                line_color=info['color'],
                annotation_text=label,
                annotation_position=positions[pos_idx % len(positions)],
                annotation_font_size=10,
                annotation_font_color=info['color']
            )
            pos_idx += 1


        # Add current price line
        if self.underlying_price:
            fig.add_hline(
                y=self.underlying_price,
                line_dash="solid",
                line_color="red",
                line_width=2,
                annotation_text=f"Current: ${self.underlying_price:.2f}",
                annotation_position="top right", # Changed from "top center" to "top right"
                annotation_font_color="red"
            )

        # Update layout
        fig.update_layout(
            title=f'QQQ with Greek Exposure Levels (Expiry: {nearest_expiry})',
            xaxis_title='Time',
            yaxis_title='Price',
            xaxis_rangeslider_visible=False,
            height=700,
            width=1200,
            showlegend=False # Legends for lines are handled by annotations
        )

        fig.show()

        # Print summary
        print(f"\nCurrent QQQ Price: ${self.underlying_price:.2f}")
        for label, info in sig_levels.items():
            print(f"{label}: ${info['strike']:.2f} (Value: {info['value']/1e9:.3f}B)")

# Usage
def main():
    # Initialize analyzer for QQQ
    analyzer = GammaExposureAnalyzer("QQQ")

    # Get available expirations
    expirations = analyzer.ticker.options

    if not expirations:
        print("No options data available for QQQ")
        return

    print(f"Available expirations: {expirations[:5]}")

    # Get current price
    current_price = analyzer.get_current_price()
    if current_price is None:
        print("Could not retrieve current QQQ price. Exiting.")
        return
    print(f"Current QQQ Price: ${current_price:.2f}")

    # Use nearest expiration
    nearest_expiry = expirations[0]
    print(f"\nAnalyzing nearest expiration: {nearest_expiry}")

    # Plot all metrics
    _ = analyzer.plot_all_metrics(nearest_expiry)

    print("\n" + "="*80)
    print("GENERATING QQQ CHART WITH GREEK LEVELS")
    print("="*80)
    analyzer.plot_qqq_with_greeks_levels()

if __name__ == "__main__":
    main()


Available expirations: ('2026-01-13', '2026-01-14', '2026-01-15', '2026-01-16', '2026-01-20')
Current QQQ Price: $628.27

Analyzing nearest expiration: 2026-01-13



QQQ OPTIONS EXPOSURE SUMMARY
Underlying Price: $628.27
Expiration: 2026-01-13

--- Open Interest Based Exposures ---
Total Net DEX:      $0.000B
Total Net GEX:      $0.000B
Total Abs GEX:      $0.000B
Total Delta-Adj Gamma: $0.000B
Total Net VEX:      $0.000B
Total Net Vanna:    $0.000B

--- Key Strike Levels ---
Strike with Max |GEX|: $505.00
Strike with Max |Vanna|: $505.00

GENERATING QQQ CHART WITH GREEK LEVELS
Analyzing nearest expiration: 2026-01-13



Current QQQ Price: $628.39
Abs Max DEX ($505.00): $505.00 (Value: 0.000B)
Abs Max GEX ($505.00): $505.00 (Value: 0.000B)
Abs Max DAG ($505.00): $505.00 (Value: 0.000B)
Abs Max VEX ($505.00): $505.00 (Value: 0.000B)
Abs Max Vanna ($505.00): $505.00 (Value: 0.000B)


In [ ]:
# @title
import yfinance as yf
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt # Removed matplotlib
from datetime import datetime, date, timedelta
import warnings
warnings.filterwarnings('ignore')

# Import Plotly libraries
import plotly.graph_objects as go
from plotly.subplots import make_subplots

class GammaExposureAnalyzer:
    def __init__(self, symbol="^SPX"):
        self.symbol = symbol
        self.ticker = yf.Ticker(symbol)
        self.underlying_price = None
        self.expiration_dates = None

    def get_current_price(self):
        """Get current underlying price"""
        hist = pd.DataFrame()
        try:
            hist = self.ticker.history(period='1d', interval='1m')
            if hist.empty:
                print("No 1-minute data, trying daily...")
                hist = self.ticker.history(period='1d')
        except Exception as e:
            print(f"Error fetching 1-minute data: {e}, trying daily...")
            hist = self.ticker.history(period='1d')

        if not hist.empty:
            self.underlying_price = hist['Close'].iloc[-1]
        else:
            self.underlying_price = None
            print(f"Warning: Could not retrieve price for {self.symbol}.")

        return self.underlying_price

    def get_options_chain(self, expiration_date):
        """Get options chain for specific expiration"""
        try:
            chain = self.ticker.option_chain(expiration_date)
            return chain.calls, chain.puts
        except Exception as e:
            print(f"Error fetching options chain: {e}")
            return None, None

    def calculate_gamma(self, option_type, strike, price, iv, days_to_expiry):
        """Calculate gamma for an option using Black-Scholes formula"""
        if days_to_expiry <= 0:
            return 0

        try:
            S = self.underlying_price
            K = strike
            r = 0.02  # risk-free rate (2%)
            T = days_to_expiry / 365.0
            sigma = iv

            if T <= 0:
                return 0

            # Calculate d1 for Black-Scholes
            d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))

            # Gamma calculation (same for calls and puts)
            gamma = np.exp(-d1**2 / 2) / (S * sigma * np.sqrt(2 * np.pi * T))

            return gamma
        except:
            return 0

    def calculate_gex(self, expiration_date):
        """Calculate Gamma Exposure for a specific expiration"""
        calls, puts = self.get_options_chain(expiration_date)

        if calls is None or puts is None:
            return None, None

        # Get current date and calculate days to expiry
        today = datetime.now()
        # Ensure expiry_date is a date object for comparison with today
        if isinstance(expiration_date, str):
            expiry_date = datetime.strptime(expiration_date, '%Y-%m-%d').date()
        else:
            expiry_date = expiration_date.date() # if it's already a datetime object

        # Calculate days to expiry. Add 1 to ensure options expiring today are considered 1 day out.
        days_to_expiry = (expiry_date - today.date()).days + 1
        if days_to_expiry <= 0:
            # Handle cases where expiry might be in the past or today for 0DTE
            days_to_expiry = 0.001 # Smallest positive number to avoid division by zero

        # Prepare DataFrames
        calls['type'] = 'CALL'
        puts['type'] = 'PUT'

        # Combine calls and puts
        all_options = pd.concat([calls, puts])

        # Calculate gamma for each option
        all_options['gamma'] = all_options.apply(
            lambda row: self.calculate_gamma(
                row['type'],
                row['strike'],
                row['lastPrice'],
                row.get('impliedVolatility', 0.3), # Default IV if not available
                days_to_expiry
            ), axis=1
        )

        # Calculate Gamma Exposure (GEX)
        # GEX = Gamma * Open Interest * 100 * Spot Price^2
        all_options['gex_oi'] = all_options['gamma'] * all_options['openInterest'] * 100 * (self.underlying_price ** 2)
        all_options['gex_volume'] = all_options['gamma'] * all_options['volume'] * 100 * (self.underlying_price ** 2)

        # Separate calls and puts
        calls = all_options[all_options['type'] == 'CALL'].copy()
        puts = all_options[all_options['type'] == 'PUT'].copy()

        # Merge based on strike to get net GEX
        merged = pd.merge(
            calls[['strike', 'gex_oi', 'gex_volume']].rename(
                columns={'gex_oi': 'call_gex_oi', 'gex_volume': 'call_gex_volume'}
            ),
            puts[['strike', 'gex_oi', 'gex_volume']].rename(
                columns={'gex_oi': 'put_gex_oi', 'gex_volume': 'put_gex_volume'}
            ),
            on='strike',
            how='outer'
        ).fillna(0)

        merged['net_gex_oi'] = merged['call_gex_oi'] - merged['put_gex_oi']
        merged['abs_gex_oi'] = merged['call_gex_oi'] + merged['put_gex_oi'] # Sum of absolute GEX, not net
        merged['net_gex_volume'] = merged['call_gex_volume'] - merged['put_gex_volume']
        merged['abs_gex_volume'] = merged['call_gex_volume'] + merged['put_gex_volume']

        return merged, all_options

    def calculate_dex(self, expiration_date):
        """Calculate Delta Exposure for a specific expiration"""
        calls, puts = self.get_options_chain(expiration_date)

        if calls is None or puts is None:
            return None

        # Get current date and calculate days to expiry
        today = datetime.now()
        # Ensure expiry_date is a date object for comparison with today
        if isinstance(expiration_date, str):
            expiry_date = datetime.strptime(expiration_date, '%Y-%m-%d').date()
        else:
            expiry_date = expiration_date.date() # if it's already a datetime object

        # Calculate days to expiry. Add 1 to ensure options expiring today are considered 1 day out.
        days_to_expiry = (expiry_date - today.date()).days + 1
        if days_to_expiry <= 0:
            days_to_expiry = 0.001 # Smallest positive number to avoid division by zero

        # Prepare DataFrames: Add 'type' column
        calls['type'] = 'CALL'
        puts['type'] = 'PUT'

        # Combine and calculate DEX
        all_options = pd.concat([calls, puts])

        # Simplified delta calculation
        # For deep ITM calls: delta ~ 1, for deep OTM calls: delta ~ 0
        # For deep ITM puts: delta ~ -1, for deep OTM puts: delta ~ 0
        def calculate_delta(option_type, strike, price):
            # Using implied volatility from yfinance data for a more accurate delta calculation
            # If IV is not available, default to a simplified model
            S = self.underlying_price
            K = strike
            r = 0.02 # Risk-free rate
            T = days_to_expiry / 365.0

            # If impliedVolatility is available in the option chain, use it.
            # Otherwise, use a simplified approach or a default.
            # For now, let's stick to the simplified method as in the original code, but note the enhancement possibility.
            moneyness = S / K - 1
            if option_type == 'CALL':
                if moneyness > 0.05:  # ITM
                    return 0.7 + 0.3 * min(moneyness / 0.5, 1)
                elif moneyness < -0.05:  # OTM
                    return 0.3 * max(1 + moneyness / 0.5, 0)
                else:  # ATM
                    return 0.5
            else:  # PUT
                if moneyness < -0.05:  # ITM
                    return -0.7 - 0.3 * min(abs(moneyness) / 0.5, 1)
                elif moneyness > 0.05:  # OTM
                    return -0.3 * max(1 - moneyness / 0.5, 0)
                else:  # ATM
                    return -0.5

        all_options['delta'] = all_options.apply(
            lambda row: calculate_delta(
                'CALL' if row['type'] == 'CALL' else 'PUT',
                row['strike'],
                row['lastPrice']
            ), axis=1
        )

        # Calculate Delta Exposure (DEX)
        all_options['dex_oi'] = all_options['delta'] * all_options['openInterest'] * 100 * self.underlying_price
        all_options['dex_volume'] = all_options['delta'] * all_options['volume'] * 100 * self.underlying_price

        # Separate and aggregate
        calls = all_options[all_options['type'] == 'CALL'][['strike', 'dex_oi', 'dex_volume']]
        puts = all_options[all_options['type'] == 'PUT'][['strike', 'dex_oi', 'dex_volume']]

        merged = pd.merge(
            calls.rename(columns={'dex_oi': 'call_dex_oi', 'dex_volume': 'call_dex_volume'}),
            puts.rename(columns={'dex_oi': 'put_dex_oi', 'dex_volume': 'put_dex_volume'}),
            on='strike',
            how='outer'
        ).fillna(0)

        merged['net_dex_oi'] = merged['call_dex_oi'] + merged['put_dex_oi']  # Note: put deltas are negative
        merged['abs_dex_oi'] = abs(merged['call_dex_oi']) + abs(merged['put_dex_oi'])
        merged['net_dex_volume'] = merged['call_dex_volume'] + merged['put_dex_volume']
        merged['abs_dex_volume'] = abs(merged['call_dex_volume']) + abs(merged['put_dex_volume'])

        return merged

    def plot_all_metrics(self, expiration_date):
        """Create comprehensive plots of all GEX and DEX metrics"""
        # Get current price
        self.get_current_price()

        # Calculate DEX and GEX
        dex_df = self.calculate_dex(expiration_date)
        gex_df, all_options = self.calculate_gex(expiration_date)

        if dex_df is None or gex_df is None:
            print("Failed to fetch data")
            return

        # Merge DEX and GEX data
        merged_df = pd.merge(dex_df, gex_df, on='strike', how='outer').fillna(0)

        # Filter for relevant strikes (within 20% of current price)
        if self.underlying_price is None:
            print("Underlying price not available for filtering strikes.")
            filtered_df = merged_df
        else:
            price_range = 0.20
            min_strike = self.underlying_price * (1 - price_range)
            max_strike = self.underlying_price * (1 + price_range)
            filtered_df = merged_df[(merged_df['strike'] >= min_strike) & (merged_df['strike'] <= max_strike)]

        # Create Plotly figure with subplots (5 rows, 2 columns for 10 plots)
        fig = make_subplots(
            rows=5, cols=2,
            subplot_titles=(
                'DEX (Delta Exposure) - Open Interest',
                'Net GEX (Gamma Exposure) - Open Interest',
                'Absolute GEX - Open Interest',
                'Net GEX - Volume',
                'Absolute GEX - Volume',
                'Gamma by Strike (Raw Gamma)',
                'DEX Comparison: OI vs Volume',
                'GEX Comparison: OI vs Volume',
                'Total Gamma Profile (Net GEX)',
                'Total Gamma Profile (Absolute GEX)'
            )
        )

        # Plot 1: DEX (Delta Exposure) - Open Interest
        fig.add_trace(
            go.Bar(x=filtered_df['strike'], y=filtered_df['net_dex_oi'], name='Net DEX OI', marker_color='blue'),
            row=1, col=1
        )
        fig.add_hline(y=0, line_dash="dash", line_color="black", annotation_text="Zero DEX", row=1, col=1)
        if self.underlying_price:
            fig.add_vline(x=self.underlying_price, line_dash="dash", line_color="red", annotation_text=f'Spot: ${self.underlying_price:.2f}', row=1, col=1)

        # Plot 2: Net GEX (Gamma Exposure) - Open Interest
        fig.add_trace(
            go.Bar(x=filtered_df['strike'], y=filtered_df['net_gex_oi'], name='Net GEX OI', marker_color='green'),
            row=1, col=2
        )
        fig.add_hline(y=0, line_dash="dash", line_color="black", annotation_text="Zero GEX", row=1, col=2)
        if self.underlying_price:
            fig.add_vline(x=self.underlying_price, line_dash="dash", line_color="red", row=1, col=2)

        # Plot 3: Absolute GEX (Open Interest)
        fig.add_trace(
            go.Bar(x=filtered_df['strike'], y=filtered_df['abs_gex_oi'], name='Abs GEX OI', marker_color='orange'),
            row=2, col=1
        )
        if self.underlying_price:
            fig.add_vline(x=self.underlying_price, line_dash="dash", line_color="red", row=2, col=1)

        # Plot 4: Net GEX - Volume
        fig.add_trace(
            go.Bar(x=filtered_df['strike'], y=filtered_df['net_gex_volume'], name='Net GEX Volume', marker_color='purple'),
            row=2, col=2
        )
        fig.add_hline(y=0, line_dash="dash", line_color="black", annotation_text="Zero GEX", row=2, col=2)
        if self.underlying_price:
            fig.add_vline(x=self.underlying_price, line_dash="dash", line_color="red", row=2, col=2)

        # Plot 5: Absolute GEX - Volume
        fig.add_trace(
            go.Bar(x=filtered_df['strike'], y=filtered_df['abs_gex_volume'], name='Abs GEX Volume', marker_color='brown'),
            row=3, col=1
        )
        if self.underlying_price:
            fig.add_vline(x=self.underlying_price, line_dash="dash", line_color="red", row=3, col=1)

        # Plot 6: Gamma by strike (Calls vs Puts)
        if all_options is not None:
            if self.underlying_price:
                filtered_options = all_options[(all_options['strike'] >= min_strike) &
                                             (all_options['strike'] <= max_strike)]
            else:
                filtered_options = all_options

            calls_gamma = filtered_options[filtered_options['type'] == 'CALL']
            puts_gamma = filtered_options[filtered_options['type'] == 'PUT']

            fig.add_trace(
                go.Scatter(x=calls_gamma['strike'], y=calls_gamma['gamma'], mode='markers', name='Calls Gamma', marker=dict(color='green', size=8)),
                row=3, col=2
            )
            fig.add_trace(
                go.Scatter(x=puts_gamma['strike'], y=puts_gamma['gamma'], mode='markers', name='Puts Gamma', marker=dict(color='red', size=8)),
                row=3, col=2
            )
            if self.underlying_price:
                fig.add_vline(x=self.underlying_price, line_dash="dash", line_color="red", row=3, col=2)

        # Plot 7: Combined DEX view (OI + Volume)
        fig.add_trace(
            go.Bar(x=filtered_df['strike'], y=filtered_df['net_dex_oi'], name='DEX OI', marker_color='blue', opacity=0.7),
            row=4, col=1
        )
        fig.add_trace(
            go.Bar(x=filtered_df['strike'], y=filtered_df['net_dex_volume'], name='DEX Volume', marker_color='cyan', opacity=0.7),
            row=4, col=1
        )
        fig.add_hline(y=0, line_dash="dash", line_color="black", row=4, col=1)
        if self.underlying_price:
            fig.add_vline(x=self.underlying_price, line_dash="dash", line_color="red", row=4, col=1)

        # Plot 8: Combined GEX view (OI + Volume)
        fig.add_trace(
            go.Bar(x=filtered_df['strike'], y=filtered_df['net_gex_oi'], name='GEX OI', marker_color='green', opacity=0.7),
            row=4, col=2
        )
        fig.add_trace(
            go.Bar(x=filtered_df['strike'], y=filtered_df['net_gex_volume'], name='GEX Volume', marker_color='lightgreen', opacity=0.7),
            row=4, col=2
        )
        fig.add_hline(y=0, line_dash="dash", line_color="black", row=4, col=2)
        if self.underlying_price:
            fig.add_vline(x=self.underlying_price, line_dash="dash", line_color="red", row=4, col=2)

        # Plot 9: Total Gamma Profile (Net GEX)
        fig.add_trace(
            go.Scatter(x=filtered_df['strike'], y=filtered_df['net_gex_oi'], mode='lines', name='Net GEX OI', line=dict(color='green', width=2)),
            row=5, col=1
        )
        fig.add_trace(
            go.Scatter(x=filtered_df['strike'], y=filtered_df['net_gex_volume'], mode='lines', name='Net GEX Volume', line=dict(color='blue', dash='dash', width=2)),
            row=5, col=1
        )
        fig.add_hline(y=0, line_dash="dash", line_color="black", row=5, col=1)
        if self.underlying_price:
            fig.add_vline(x=self.underlying_price, line_dash="dash", line_color="red", row=5, col=1)

        # Plot 10: Total Gamma Profile (Absolute GEX)
        fig.add_trace(
            go.Scatter(x=filtered_df['strike'], y=filtered_df['abs_gex_oi'], mode='lines', name='Abs GEX OI', line=dict(color='orange', width=2)),
            row=5, col=2
        )
        fig.add_trace(
            go.Scatter(x=filtered_df['strike'], y=filtered_df['abs_gex_volume'], mode='lines', name='Abs GEX Volume', line=dict(color='brown', dash='dash', width=2)),
            row=5, col=2
        )
        if self.underlying_price:
            fig.add_vline(x=self.underlying_price, line_dash="dash", line_color="red", row=5, col=2)

        # Update layout for overall title and interactive features
        fig.update_layout(
            title_text=f'SPX Options Analysis - Expiry: {expiration_date}<br>Current Price: ${self.underlying_price:.2f}' if self.underlying_price else f'SPX Options Analysis - Expiry: {expiration_date}<br>Current Price: N/A',
            height=2000, width=1200, # Adjust overall figure size for 10 plots
            showlegend=True,
            hovermode='x unified',
            legend=dict(orientation="h", yanchor="bottom", y=-0.05, xanchor="center", x=0.5)
        )

        # Update x and y axes labels for all subplots
        for i in range(1, 6):
            for j in range(1, 3):
                fig.update_xaxes(title_text='Strike Price', row=i, col=j)
                # Specific y-axis labels based on the plot
                if (i, j) in [(1,1), (2,1), (4,1)]: # DEX related
                    fig.update_yaxes(title_text='Net DEX', row=i, col=j)
                elif (i, j) in [(1,2), (2,2), (3,1), (4,2), (5,1), (5,2)]: # GEX related (including new plot)
                    fig.update_yaxes(title_text='Gamma Exposure', row=i, col=j)
                elif (i, j) == (3,2): # Raw Gamma
                    fig.update_yaxes(title_text='Gamma', row=i, col=j)

        fig.show()

        # Print summary statistics
        print("\n" + "="*60)
        print("SUMMARY STATISTICS")
        print("="*60)
        print(f"Underlying Price: ${self.underlying_price:.2f}" if self.underlying_price else "Underlying Price: N/A")
        print(f"Expiration: {expiration_date}")
        if not filtered_df.empty:
            print(f"Total Net DEX (OI): ${filtered_df['net_dex_oi'].sum()/1e9:.3f}B")
            print(f"Total Net GEX (OI): ${filtered_df['net_gex_oi'].sum()/1e9:.3f}B")
            print(f"Total Abs GEX (OI): ${filtered_df['abs_gex_oi'].sum()/1e9:.3f}B")
            print(f"Total Net GEX (Volume): ${filtered_df['net_gex_volume'].sum()/1e9:.3f}B")
            print(f"Total Abs GEX (Volume): ${filtered_df['abs_gex_volume'].sum()/1e9:.3f}B")
        else:
            print("No filtered data available for summary statistics.")

        return filtered_df

    def plot_spx_with_greeks_levels(self):
        """Fetches 1-minute SPX data, calculates max abs GEX/DEX strikes for nearest expiry,
        and plots SPX with horizontal lines at these levels.
        """
        # 1. Fetch 1-minute historical data for SPX
        spx_1min_data = pd.DataFrame()
        try:
            spx_1min_data = self.ticker.history(period='1d', interval='1m')
            if spx_1min_data.empty:
                print("No 1-minute data available for today. Falling back to daily data for candlestick plot.")
                spx_1min_data = self.ticker.history(period='1d') # Fallback for candlestick
        except Exception as e:
            print(f"Error fetching 1-minute SPX data: {e}. Falling back to daily data for candlestick plot.")
            spx_1min_data = self.ticker.history(period='1d') # Fallback for candlestick

        # Ensure underlying_price is set for calculations (it will handle its own empty data)
        self.get_current_price()

        if self.underlying_price is None:
            print("Cannot plot Greek levels without a current underlying price.")
            return

        # 2. Retrieve available option expiration dates and select nearest expiry
        expirations = self.ticker.options
        if not expirations:
            print("No options data available.")
            return

        nearest_expiry = expirations[0]
        print(f"Analyzing nearest expiration for Greek levels: {nearest_expiry}")

        # 3. Calculate DEX and GEX for nearest expiry
        dex_df = self.calculate_dex(nearest_expiry)
        gex_df, _ = self.calculate_gex(nearest_expiry)

        if dex_df is None or gex_df is None:
            print("Failed to calculate DEX or GEX for the nearest expiration.")
            return

        # Filter for relevant strikes (within 20% of current price)
        price_range = 0.20
        min_strike = self.underlying_price * (1 - price_range)
        max_strike = self.underlying_price * (1 + price_range)

        # Merge DEX and GEX data for easier filtering and analysis
        merged_df = pd.merge(dex_df, gex_df, on='strike', how='outer').fillna(0)
        filtered_df = merged_df[(merged_df['strike'] >= min_strike) & (merged_df['strike'] <= max_strike)]

        # 4. Identify significant strike prices
        max_dex_strike = None
        max_gex_oi_strike = None
        max_gex_volume_strike = None

        max_pos_net_gex_oi_strike = None
        min_neg_net_gex_oi_strike = None
        max_pos_net_gex_volume_strike = None
        min_neg_net_gex_volume_strike = None

        if not filtered_df.empty:
            # Max Abs DEX OI
            if 'abs_dex_oi' in filtered_df.columns and not filtered_df['abs_dex_oi'].isnull().all():
                max_dex_strike = filtered_df.loc[filtered_df['abs_dex_oi'].idxmax()]['strike']

            # Max Abs GEX OI
            if 'abs_gex_oi' in filtered_df.columns and not filtered_df['abs_gex_oi'].isnull().all():
                max_gex_oi_strike = filtered_df.loc[filtered_df['abs_gex_oi'].idxmax()]['strike']

            # Max Abs GEX Volume
            if 'abs_gex_volume' in filtered_df.columns and not filtered_df['abs_gex_volume'].isnull().all():
                max_gex_volume_strike = filtered_df.loc[filtered_df['abs_gex_volume'].idxmax()]['strike']

            # Net GEX OI (positive and negative peaks)
            positive_net_gex_oi = filtered_df[filtered_df['net_gex_oi'] > 0]
            if not positive_net_gex_oi.empty and not positive_net_gex_oi['net_gex_oi'].isnull().all():
                max_pos_net_gex_oi_strike = positive_net_gex_oi.loc[positive_net_gex_oi['net_gex_oi'].idxmax()]['strike']

            negative_net_gex_oi = filtered_df[filtered_df['net_gex_oi'] < 0]
            if not negative_net_gex_oi.empty and not negative_net_gex_oi['net_gex_oi'].isnull().all():
                min_neg_net_gex_oi_strike = negative_net_gex_oi.loc[negative_net_gex_oi['net_gex_oi'].idxmin()]['strike']

            # Net GEX Volume (positive and negative peaks)
            positive_net_gex_volume = filtered_df[filtered_df['net_gex_volume'] > 0]
            if not positive_net_gex_volume.empty and not positive_net_gex_volume['net_gex_volume'].isnull().all():
                max_pos_net_gex_volume_strike = positive_net_gex_volume.loc[positive_net_gex_volume['net_gex_volume'].idxmax()]['strike']

            negative_net_gex_volume = filtered_df[filtered_df['net_gex_volume'] < 0]
            if not negative_net_gex_volume.empty and not negative_net_gex_volume['net_gex_volume'].isnull().all():
                min_neg_net_gex_volume_strike = negative_net_gex_volume.loc[negative_net_gex_volume['net_gex_volume'].idxmin()]['strike']


        print(f"Current SPX Price: {self.underlying_price:.2f}")
        if max_dex_strike: print(f"Strike with Max Abs DEX OI: {max_dex_strike:.2f}")
        if max_gex_oi_strike: print(f"Strike with Max Abs GEX OI: {max_gex_oi_strike:.2f}")
        if max_gex_volume_strike: print(f"Strike with Max Abs GEX Volume: {max_gex_volume_strike:.2f}")
        if max_pos_net_gex_oi_strike: print(f"Strike with Max Pos Net GEX OI: {max_pos_net_gex_oi_strike:.2f}")
        if min_neg_net_gex_oi_strike: print(f"Strike with Max Neg Net GEX OI: {min_neg_net_gex_oi_strike:.2f}")
        if max_pos_net_gex_volume_strike: print(f"Strike with Max Pos Net GEX Volume: {max_pos_net_gex_volume_strike:.2f}")
        if min_neg_net_gex_volume_strike: print(f"Strike with Max Neg Net GEX Volume: {min_neg_net_gex_volume_strike:.2f}")


        # 5. Create Plotly figure
        fig = go.Figure()

        # 6. Add candlestick trace if 1-minute data is available and meaningful
        if not spx_1min_data.empty and 'Open' in spx_1min_data.columns:
            fig.add_trace(go.Candlestick(
                x=spx_1min_data.index,
                open=spx_1min_data['Open'],
                high=spx_1min_data['High'],
                low=spx_1min_data['Low'],
                close=spx_1min_data['Close'],
                name='SPX Candlestick',
                increasing_line_color='green',
                decreasing_line_color='red'
            ))
        else:
            print("1-minute SPX data is not suitable for candlestick plot.")
            # If no 1-min data, plot current price as a dot
            if self.underlying_price:
                 fig.add_trace(go.Scatter(
                    x=[datetime.now()],
                    y=[self.underlying_price],
                    mode='markers',
                    marker=dict(size=10, color='blue'),
                    name='Current SPX Price' # Legend will show for this if candlestick is not plotted.
                 ))

        # 7. Add horizontal lines for significant Greek levels
        if max_dex_strike:
            fig.add_hline(y=max_dex_strike, line_dash="dash", line_color="blue",
                          annotation_text=f'Max Abs DEX OI: {max_dex_strike:.2f}',
                          annotation_position="bottom right",
                          annotation_font_color="blue")
        if max_gex_oi_strike:
            fig.add_hline(y=max_gex_oi_strike, line_dash="dot", line_color="green",
                          annotation_text=f'Max Abs GEX OI: {max_gex_oi_strike:.2f}',
                          annotation_position="top right",
                          annotation_font_color="green")
        if max_gex_volume_strike:
            fig.add_hline(y=max_gex_volume_strike, line_dash="dot", line_color="darkgreen",
                          annotation_text=f'Max Abs GEX Vol: {max_gex_volume_strike:.2f}',
                          annotation_position="bottom left",
                          annotation_font_color="darkgreen")

        if max_pos_net_gex_oi_strike:
            fig.add_hline(y=max_pos_net_gex_oi_strike, line_dash="dashdot", line_color="orange",
                          annotation_text=f'Max Pos Net GEX OI: {max_pos_net_gex_oi_strike:.2f}',
                          annotation_position="top left",
                          annotation_font_color="orange")
        if min_neg_net_gex_oi_strike:
            fig.add_hline(y=min_neg_net_gex_oi_strike, line_dash="dashdot", line_color="purple",
                          annotation_text=f'Max Neg Net GEX OI: {min_neg_net_gex_oi_strike:.2f}',
                          annotation_position="bottom left",
                          annotation_font_color="purple")
        if max_pos_net_gex_volume_strike:
            fig.add_hline(y=max_pos_net_gex_volume_strike, line_dash="longdash", line_color="brown",
                          annotation_text=f'Max Pos Net GEX Vol: {max_pos_net_gex_volume_strike:.2f}',
                          annotation_position="top right",
                          annotation_font_color="brown")
        if min_neg_net_gex_volume_strike:
            fig.add_hline(y=min_neg_net_gex_volume_strike, line_dash="longdash", line_color="pink",
                          annotation_text=f'Max Neg Net GEX Vol: {min_neg_net_gex_volume_strike:.2f}',
                          annotation_position="bottom right",
                          annotation_font_color="pink")

        # 8. Add a horizontal line for the current self.underlying_price
        if self.underlying_price:
            fig.add_hline(y=self.underlying_price, line_dash="solid", line_color="red",
                          annotation_text=f'Current SPX Price: {self.underlying_price:.2f}',
                          annotation_position="top left",
                          annotation_font_color="red")

        # 9. Set appropriate title and labels
        fig.update_layout(
            title_text=f'SPX 1-Minute Chart with Greek Levels (Expiry: {nearest_expiry})',
            xaxis_title='Time',
            yaxis_title='Price',
            xaxis_rangeslider_visible=False, # Hide range slider for better view
            height=700, width=1000,
            showlegend=False # Legends for lines are handled by annotations
        )

        # 10. Display the plot
        fig.show()

# Usage
def main():
    # Initialize analyzer
    analyzer = GammaExposureAnalyzer("^SPX")

    # Get available expirations
    # The line below is redundant as it's called inside the new method anyway
    # analyzer.ticker = yf.Ticker("^SPX")
    expirations = analyzer.ticker.options

    if not expirations:
        print("No options data available for SPX")
        return

    print(f"Available expirations: {expirations[:5]}")

    # Get current price
    current_price = analyzer.get_current_price()
    if current_price is None:
        print("Could not retrieve current SPX price. Exiting.")
        return
    print(f"Current SPX Price: ${current_price:.2f}")

    # Use nearest expiration for plot_all_metrics
    nearest_expiry = expirations[0]
    print(f"\nAnalyzing nearest expiration for full metrics: {nearest_expiry}")

    # Plot all metrics
    _ = analyzer.plot_all_metrics(nearest_expiry)

    print("\n" + "="*60)
    print("GENERATING SPX CHART WITH GREEK LEVELS")
    print("="*60)
    analyzer.plot_spx_with_greeks_levels()

if __name__ == "__main__":
    main()

Available expirations: ('2026-01-13', '2026-01-14', '2026-01-15', '2026-01-16', '2026-01-20')
Current SPX Price: $6979.53

Analyzing nearest expiration for full metrics: 2026-01-13



SUMMARY STATISTICS
Underlying Price: $6979.53
Expiration: 2026-01-13
Total Net DEX (OI): $-5.345B
Total Net GEX (OI): $474.080B
Total Abs GEX (OI): $735.368B
Total Net GEX (Volume): $849.335B
Total Abs GEX (Volume): $2753.730B

GENERATING SPX CHART WITH GREEK LEVELS
Analyzing nearest expiration for Greek levels: 2026-01-13
Current SPX Price: 6979.53
Strike with Max Abs DEX OI: 7075.00
Strike with Max Abs GEX OI: 7000.00
Strike with Max Abs GEX Volume: 7000.00
Strike with Max Pos Net GEX OI: 7000.00
Strike with Max Neg Net GEX OI: 6945.00
Strike with Max Pos Net GEX Volume: 7000.00
Strike with Max Neg Net GEX Volume: 6965.00


In [ ]:
# @title
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import datetime

# Initialize analyzer for SPX
analyzer = GammaExposureAnalyzer("^SPX")

# Get current price
current_price = analyzer.get_current_price()
if current_price is None:
    print("Could not retrieve current SPX price. Exiting.")
else:
    print(f"Current SPX Price: ${current_price:.2f}")

# Get nearest expiration
expirations = analyzer.ticker.options
if not expirations:
    print("No options data available for SPX")
else:
    nearest_expiry = expirations[0]
    print(f"Analyzing nearest expiration for average GEX per strike: {nearest_expiry}")

    # Calculate GEX
    gex_df_tuple = analyzer.calculate_gex(nearest_expiry)
    gex_df = gex_df_tuple[0] if gex_df_tuple else None

    if gex_df is None or gex_df.empty:
        print("Failed to fetch GEX data for SPX.")
    else:
        # Filter for relevant strikes (within 20% of current price)
        if current_price:
            price_range = 0.20
            min_strike = current_price * (1 - price_range)
            max_strike = current_price * (1 + price_range)
            filtered_df = gex_df[(gex_df['strike'] >= min_strike) & (gex_df['strike'] <= max_strike)]
        else:
            filtered_df = gex_df

        if filtered_df.empty:
            print("No relevant GEX data after filtering.")
        else:
            # Calculate the average of net_gex_oi and net_gex_volume for each strike
            filtered_df['avg_net_gex_per_strike'] = (filtered_df['net_gex_oi'] + filtered_df['net_gex_volume']) / 2

            # Calculate the average of abs_gex_oi and abs_gex_volume for each strike
            filtered_df['avg_abs_gex_per_strike'] = (filtered_df['abs_gex_oi'] + filtered_df['abs_gex_volume']) / 2

            # Plotting the average Net GEX per strike
            fig_net = go.Figure()
            fig_net.add_trace(
                go.Bar(x=filtered_df['strike'], y=filtered_df['avg_net_gex_per_strike'],
                      name='Average Net GEX (OI & Volume)', marker_color='green')
            )
            fig_net.add_hline(y=0, line_dash="dash", line_color="black")
            if current_price:
                fig_net.add_vline(x=current_price, line_dash="dash", line_color="red", annotation_text=f'Spot: ${current_price:.2f}')

            fig_net.update_layout(
                title_text=f'SPX Average Net GEX Per Strike (OI & Volume) - Expiry: {nearest_expiry}',
                xaxis_title='Strike Price',
                yaxis_title='Average Net GEX Exposure',
                height=300, width=1000,
                hovermode='x unified'
            )
            fig_net.show()

            # Plotting the average Absolute GEX per strike
            fig_abs = go.Figure()
            fig_abs.add_trace(
                go.Bar(x=filtered_df['strike'], y=filtered_df['avg_abs_gex_per_strike'],
                      name='Average Absolute GEX (OI & Volume)', marker_color='orange')
            )
            if current_price:
                fig_abs.add_vline(x=current_price, line_dash="dash", line_color="red", annotation_text=f'Spot: ${current_price:.2f}')

            fig_abs.update_layout(
                title_text=f'SPX Average Absolute GEX Per Strike (OI & Volume) - Expiry: {nearest_expiry}',
                xaxis_title='Strike Price',
                yaxis_title='Average Absolute GEX Exposure',
                height=300, width=1000,
                hovermode='x unified'
            )
            fig_abs.show()

            # # Print summary statistics
            # print("\n" + "="*80)
            # print(f"SPX GEX Per Strike Summaries for {nearest_expiry}")
            # print("="*80)
            # print(f"Underlying Price: ${current_price:.2f}" if current_price else "Underlying Price: N/A")
            # print(f"Expiration: {nearest_expiry}")

            # print("\n--- Average Net GEX per Strike ---")
            # print(f"Total Average Net GEX across all strikes: ${filtered_df['avg_net_gex_per_strike'].sum()/1e9:.3f}B")
            # if not filtered_df.empty and 'avg_net_gex_per_strike' in filtered_df.columns and not filtered_df['avg_net_gex_per_strike'].isnull().all():
            #     max_avg_net_gex_strike = filtered_df.loc[filtered_df['avg_net_gex_per_strike'].abs().idxmax(), 'strike']
            #     print(f"Strike with Max Absolute Average Net GEX: ${max_avg_net_gex_strike:.2f}")
            # else:
            #     print("No filtered data available for Average Net GEX summary.")

            # print("\n--- Average Absolute GEX per Strike ---")
            # print(f"Total Average Absolute GEX across all strikes: ${filtered_df['avg_abs_gex_per_strike'].sum()/1e9:.3f}B")
            # if not filtered_df.empty and 'avg_abs_gex_per_strike' in filtered_df.columns and not filtered_df['avg_abs_gex_per_strike'].isnull().all():
            #     max_avg_abs_gex_strike = filtered_df.loc[filtered_df['avg_abs_gex_per_strike'].idxmax(), 'strike']
            #     print(f"Strike with Max Average Absolute GEX: ${max_avg_abs_gex_strike:.2f}")
            # else:
            #     print("No filtered data available for Average Absolute GEX summary.")

Current SPX Price: $6979.53
Analyzing nearest expiration for average GEX per strike: 2026-01-13


In [ ]:
# @title
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import datetime

# Initialize analyzer for SPX
analyzer = GammaExposureAnalyzer("^SPX")

# Get current price
current_price = analyzer.get_current_price()
if current_price is None:
    print("Could not retrieve current SPX price. Exiting.")
else:
    print(f"Current SPX Price: ${current_price:.2f}")

# Get nearest expiration
expirations = analyzer.ticker.options
if not expirations:
    print("No options data available for SPX")
else:
    nearest_expiry = expirations[0]
    print(f"Analyzing nearest expiration for GEX ratio: {nearest_expiry}")

    # Calculate GEX
    gex_df_tuple = analyzer.calculate_gex(nearest_expiry)
    gex_df = gex_df_tuple[0] if gex_df_tuple else None

    if gex_df is None or gex_df.empty:
        print("Failed to fetch GEX data for SPX.")
    else:
        # Calculate GEX Ratio (Net GEX OI / Net GEX Volume)
        # Handle division by zero: replace 0 with NaN first, then replace inf with NaN, then fill NaN with 0
        gex_df['net_gex_volume_safe'] = gex_df['net_gex_volume'].replace(0, np.nan)
        gex_df['gex_ratio'] = gex_df['net_gex_oi'] / gex_df['net_gex_volume_safe']
        gex_df['gex_ratio'] = gex_df['gex_ratio'].replace([np.inf, -np.inf], np.nan).fillna(0)

        # Filter for relevant strikes (within 20% of current price)
        if current_price:
            price_range = 0.20
            min_strike = current_price * (1 - price_range)
            max_strike = current_price * (1 + price_range)
            filtered_df = gex_df[(gex_df['strike'] >= min_strike) & (gex_df['strike'] <= max_strike)].copy()
        else:
            filtered_df = gex_df.copy()

        if filtered_df.empty:
            print("No relevant GEX data after filtering.")
        else:
            # Plotting the GEX Ratio
            fig = go.Figure()
            fig.add_trace(
                go.Bar(x=filtered_df['strike'], y=filtered_df['gex_ratio'],
                      name='GEX Ratio (OI/Volume)', marker_color='darkblue')
            )
            fig.add_hline(y=0, line_dash="dash", line_color="black")
            if current_price:
                fig.add_vline(x=current_price, line_dash="dash", line_color="red", annotation_text=f'Spot: ${current_price:.2f}')

            fig.update_layout(
                title_text=f'SPX GEX Ratio (Net GEX OI / Net GEX Volume) - Expiry: {nearest_expiry}',
                xaxis_title='Strike Price',
                yaxis_title='GEX Ratio (OI/Volume)',
                height=300, width=1000,
                hovermode='x unified'
            )
            fig.show()

            # # Summary Statistics
            # print("\n" + "="*80)
            # print(f"SPX GEX Ratio (Net GEX OI / Net GEX Volume) Summary for {nearest_expiry}")
            # print("="*80)
            # print(f"Underlying Price: ${current_price:.2f}" if current_price else "Underlying Price: N/A")
            # print(f"Expiration: {nearest_expiry}")

            # positive_ratios = filtered_df[filtered_df['gex_ratio'] > 0]['gex_ratio']
            # negative_ratios = filtered_df[filtered_df['gex_ratio'] < 0]['gex_ratio']

            # print(f"\nTotal Number of Strikes Analyzed: {len(filtered_df)}")
            # print(f"Number of Strikes with Positive GEX Ratio: {len(positive_ratios)}")
            # print(f"Number of Strikes with Negative GEX Ratio: {len(negative_ratios)}")

            # if not filtered_df.empty:
            #     print(f"\nMean GEX Ratio: {filtered_df['gex_ratio'].mean():.4f}")
            #     print(f"Median GEX Ratio: {filtered_df['gex_ratio'].median():.4f}")
            #     print(f"Max GEX Ratio: {filtered_df['gex_ratio'].max():.4f} (Strike: ${filtered_df.loc[filtered_df['gex_ratio'].idxmax(), 'strike']:.2f})")
            #     print(f"Min GEX Ratio: {filtered_df['gex_ratio'].min():.4f} (Strike: ${filtered_df.loc[filtered_df['gex_ratio'].idxmin(), 'strike']:.2f})")
            #     print(f"Standard Deviation of GEX Ratio: {filtered_df['gex_ratio'].std():.4f}")
            # else:
            #     print("No data to calculate summary statistics.")

Current SPX Price: $6979.53
Analyzing nearest expiration for GEX ratio: 2026-01-13


In [ ]:
# @title
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import datetime

# Initialize analyzer for SPX
analyzer = GammaExposureAnalyzer("^SPX")

# Get current price
current_price = analyzer.get_current_price()
if current_price is None:
    print("Could not retrieve current SPX price. Exiting.")
else:
    print(f"Current SPX Price: ${current_price:.2f}")

# Get nearest expiration
expirations = analyzer.ticker.options
if not expirations:
    print("No options data available for SPX")
else:
    nearest_expiry = expirations[0]
    print(f"Analyzing nearest expiration for GEX ratios: {nearest_expiry}")

    # Calculate GEX
    gex_df_tuple = analyzer.calculate_gex(nearest_expiry)
    gex_df = gex_df_tuple[0] if gex_df_tuple else None

    if gex_df is None or gex_df.empty:
        print("Failed to fetch GEX data for SPX.")
    else:
        # Filter for relevant strikes (within 20% of current price)
        if current_price:
            price_range = 0.20
            min_strike = current_price * (1 - price_range)
            max_strike = current_price * (1 + price_range)
            filtered_df_base = gex_df[(gex_df['strike'] >= min_strike) & (gex_df['strike'] <= max_strike)].copy()
        else:
            filtered_df_base = gex_df.copy()

        if filtered_df_base.empty:
            print("No relevant GEX data after filtering.")
        else:
            # Define the ratios to calculate and plot
            ratios_to_analyze = [
                {'numerator': 'net_gex_oi', 'denominator': 'abs_gex_oi', 'name': 'Net GEX OI / Abs GEX OI', 'title_prefix': 'SPX GEX Ratio: Net OI / Abs OI'},
                {'numerator': 'net_gex_volume', 'denominator': 'abs_gex_volume', 'name': 'Net GEX Volume / Abs GEX Volume', 'title_prefix': 'SPX GEX Ratio: Net Volume / Abs Volume'},
                {'numerator': 'net_gex_oi', 'denominator': 'abs_gex_volume', 'name': 'Net GEX OI / Abs GEX Volume', 'title_prefix': 'SPX GEX Ratio: Net OI / Abs Volume'}
            ]

            for ratio_info in ratios_to_analyze:
                numerator_col = ratio_info['numerator']
                denominator_col = ratio_info['denominator']
                ratio_name = ratio_info['name']
                title_prefix = ratio_info['title_prefix']
                ratio_col = ratio_name.replace(' ', '_').replace('/', '_') # Create a safe column name

                filtered_df = filtered_df_base.copy()

                # Handle division by zero: replace 0 with NaN in denominator, then calculate, replace inf with NaN, then fill NaN with 0
                filtered_df[f'{denominator_col}_safe'] = filtered_df[denominator_col].replace(0, np.nan)
                filtered_df[ratio_col] = filtered_df[numerator_col] / filtered_df[f'{denominator_col}_safe']
                filtered_df[ratio_col] = filtered_df[ratio_col].replace([np.inf, -np.inf], np.nan).fillna(0)

                # Plotting the GEX Ratio
                fig = go.Figure()
                fig.add_trace(
                    go.Bar(x=filtered_df['strike'], y=filtered_df[ratio_col],
                          name=ratio_name, marker_color='darkblue')
                )
                fig.add_hline(y=0, line_dash="dash", line_color="black")
                if current_price:
                    fig.add_vline(x=current_price, line_dash="dash", line_color="red", annotation_text=f'Spot: ${current_price:.2f}')

                fig.update_layout(
                    title_text=f'{title_prefix} - Expiry: {nearest_expiry}',
                    xaxis_title='Strike Price',
                    yaxis_title=ratio_name,
                    height=300, width=1000,
                    hovermode='x unified'
                )
                fig.show()

                # # Summary Statistics
                # print("\n" + "="*80)
                # print(f"{title_prefix} Summary for {nearest_expiry}")
                # print("="*80)
                # print(f"Underlying Price: ${current_price:.2f}" if current_price else "Underlying Price: N/A")
                # print(f"Expiration: {nearest_expiry}")

                # positive_ratios = filtered_df[filtered_df[ratio_col] > 0][ratio_col]
                # negative_ratios = filtered_df[filtered_df[ratio_col] < 0][ratio_col]

                # print(f"\nTotal Number of Strikes Analyzed: {len(filtered_df)}")
                # print(f"Number of Strikes with Positive {ratio_name}: {len(positive_ratios)}")
                # print(f"Number of Strikes with Negative {ratio_name}: {len(negative_ratios)}")

                # if not filtered_df.empty:
                #     print(f"\nMean {ratio_name}: {filtered_df[ratio_col].mean():.4f}")
                #     print(f"Median {ratio_name}: {filtered_df[ratio_col].median():.4f}")
                #     print(f"Max {ratio_name}: {filtered_df[ratio_col].max():.4f} (Strike: ${filtered_df.loc[filtered_df[ratio_col].idxmax(), 'strike']:.2f})")
                #     print(f"Min {ratio_name}: {filtered_df[ratio_col].min():.4f} (Strike: ${filtered_df.loc[filtered_df[ratio_col].idxmin(), 'strike']:.2f})")
                #     print(f"Standard Deviation of {ratio_name}: {filtered_df[ratio_col].std():.4f}")
                # else:
                #     print("No data to calculate summary statistics.")

Current SPX Price: $6979.53
Analyzing nearest expiration for GEX ratios: 2026-01-13


In [ ]:
# @title
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import datetime

# Initialize analyzer for SPX
analyzer = GammaExposureAnalyzer("^SPX")

# Get current price
current_price = analyzer.get_current_price()
if current_price is None:
    print("Could not retrieve current SPX price. Exiting.")
else:
    print(f"Current SPX Price: ${current_price:.2f}")

# Get nearest expiration
expirations = analyzer.ticker.options
if not expirations:
    print("No options data available for SPX")
else:
    nearest_expiry = expirations[0]
    print(f"Analyzing nearest expiration for average GEX: {nearest_expiry}")

    # Calculate GEX
    gex_df_tuple = analyzer.calculate_gex(nearest_expiry)
    gex_df = gex_df_tuple[0] if gex_df_tuple else None

    if gex_df is None or gex_df.empty:
        print("Failed to fetch GEX data for SPX.")
    else:
        # Filter for relevant strikes (within 20% of current price)
        if current_price:
            price_range = 0.20
            min_strike = current_price * (1 - price_range)
            max_strike = current_price * (1 + price_range)
            filtered_df = gex_df[(gex_df['strike'] >= min_strike) & (gex_df['strike'] <= max_strike)]
        else:
            filtered_df = gex_df

        if filtered_df.empty:
            print("No relevant GEX data after filtering.")
        else:
            # Calculate the average of net_gex_oi and net_gex_volume
            avg_net_gex_oi = filtered_df['net_gex_oi'].mean()
            avg_net_gex_volume = filtered_df['net_gex_volume'].mean()

            # Create a DataFrame for plotting averages
            avg_gex_data = pd.DataFrame({
                'Metric': ['Average Net GEX (Open Interest)', 'Average Net GEX (Volume)'] ,
                'Value': [avg_net_gex_oi, avg_net_gex_volume]
            })

            # Plotting the averages
            fig = go.Figure()
            fig.add_trace(
                go.Bar(x=avg_gex_data['Metric'], y=avg_gex_data['Value'],
                      marker_color=['green', 'purple'])
            )
            fig.add_hline(y=0, line_dash="dash", line_color="black")

            fig.update_layout(
                title_text=f'SPX Average Net GEX (OI and Volume) - Expiry: {nearest_expiry}',
                xaxis_title='Metric',
                yaxis_title='Average Net GEX Exposure',
                height=400, width=800,
                hovermode='x unified'
            )
            fig.show()

            # Print summary statistics
            print("\n" + "="*80)
            print(f"SPX Average Net GEX Summary for {nearest_expiry}")
            print("="*80)
            print(f"Underlying Price: ${current_price:.2f}" if current_price else "Underlying Price: N/A")
            print(f"Expiration: {nearest_expiry}")
            print(f"Average Net GEX (Open Interest): ${avg_net_gex_oi/1e9:.3f}B")
            print(f"Average Net GEX (Volume): ${avg_net_gex_volume/1e9:.3f}B")

Current SPX Price: $6979.53
Analyzing nearest expiration for average GEX: 2026-01-13



SPX Average Net GEX Summary for 2026-01-13
Underlying Price: $6979.53
Expiration: 2026-01-13
Average Net GEX (Open Interest): $2.426B
Average Net GEX (Volume): $4.154B


In [ ]:
# @title
import yfinance as yf
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt # Removed matplotlib
from datetime import datetime, date, timedelta
import warnings
warnings.filterwarnings('ignore')

# Import Plotly libraries
import plotly.graph_objects as go
from plotly.subplots import make_subplots

class GammaExposureAnalyzer:
    def __init__(self, symbol="SPY"):
        self.symbol = symbol
        self.ticker = yf.Ticker(symbol)
        self.underlying_price = None
        self.expiration_dates = None

    def get_current_price(self):
        """Get current underlying price"""
        hist = pd.DataFrame()
        try:
            hist = self.ticker.history(period='1d', interval='1m')
            if hist.empty:
                print("No 1-minute data, trying daily...")
                hist = self.ticker.history(period='1d')
        except Exception as e:
            print(f"Error fetching 1-minute data: {e}, trying daily...")
            hist = self.ticker.history(period='1d')

        if not hist.empty:
            self.underlying_price = hist['Close'].iloc[-1]
        else:
            self.underlying_price = None
            print(f"Warning: Could not retrieve price for {self.symbol}.")

        return self.underlying_price

    def get_options_chain(self, expiration_date):
        """Get options chain for specific expiration"""
        try:
            chain = self.ticker.option_chain(expiration_date)
            return chain.calls, chain.puts
        except Exception as e:
            print(f"Error fetching options chain: {e}")
            return None, None

    def calculate_gamma(self, option_type, strike, price, iv, days_to_expiry):
        """Calculate gamma for an option using Black-Scholes formula"""
        if days_to_expiry <= 0:
            return 0

        try:
            S = self.underlying_price
            K = strike
            r = 0.02  # risk-free rate (2%)
            T = days_to_expiry / 365.0
            sigma = iv

            if T <= 0:
                return 0

            # Calculate d1 for Black-Scholes
            d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))

            # Gamma calculation (same for calls and puts)
            gamma = np.exp(-d1**2 / 2) / (S * sigma * np.sqrt(2 * np.pi * T))

            return gamma
        except:
            return 0

    def calculate_gex(self, expiration_date):
        """Calculate Gamma Exposure for a specific expiration"""
        calls, puts = self.get_options_chain(expiration_date)

        if calls is None or puts is None:
            return None, None

        # Get current date and calculate days to expiry
        today = datetime.now()
        # Ensure expiry_date is a date object for comparison with today
        if isinstance(expiration_date, str):
            expiry_date = datetime.strptime(expiration_date, '%Y-%m-%d').date()
        else:
            expiry_date = expiration_date.date() # if it's already a datetime object

        # Calculate days to expiry. Add 1 to ensure options expiring today are considered 1 day out.
        days_to_expiry = (expiry_date - today.date()).days + 1
        if days_to_expiry <= 0:
            # Handle cases where expiry might be in the past or today for 0DTE
            days_to_expiry = 0.001 # Smallest positive number to avoid division by zero

        # Prepare DataFrames
        calls['type'] = 'CALL'
        puts['type'] = 'PUT'

        # Combine calls and puts
        all_options = pd.concat([calls, puts])

        # Calculate gamma for each option
        all_options['gamma'] = all_options.apply(
            lambda row: self.calculate_gamma(
                row['type'],
                row['strike'],
                row['lastPrice'],
                row.get('impliedVolatility', 0.3), # Default IV if not available
                days_to_expiry
            ), axis=1
        )

        # Calculate Gamma Exposure (GEX)
        # GEX = Gamma * Open Interest * 100 * Spot Price^2
        all_options['gex_oi'] = all_options['gamma'] * all_options['openInterest'] * 100 * (self.underlying_price ** 2)
        all_options['gex_volume'] = all_options['gamma'] * all_options['volume'] * 100 * (self.underlying_price ** 2)

        # Separate calls and puts
        calls = all_options[all_options['type'] == 'CALL'].copy()
        puts = all_options[all_options['type'] == 'PUT'].copy()

        # Merge based on strike to get net GEX
        merged = pd.merge(
            calls[['strike', 'gex_oi', 'gex_volume']].rename(
                columns={'gex_oi': 'call_gex_oi', 'gex_volume': 'call_gex_volume'}
            ),
            puts[['strike', 'gex_oi', 'gex_volume']].rename(
                columns={'gex_oi': 'put_gex_oi', 'gex_volume': 'put_gex_volume'}
            ),
            on='strike',
            how='outer'
        ).fillna(0)

        merged['net_gex_oi'] = merged['call_gex_oi'] - merged['put_gex_oi']
        merged['abs_gex_oi'] = merged['call_gex_oi'] + merged['put_gex_oi'] # Sum of absolute GEX, not net
        merged['net_gex_volume'] = merged['call_gex_volume'] - merged['put_gex_volume']
        merged['abs_gex_volume'] = merged['call_gex_volume'] + merged['put_gex_volume']

        return merged, all_options

    def calculate_dex(self, expiration_date):
        """Calculate Delta Exposure for a specific expiration"""
        calls, puts = self.get_options_chain(expiration_date)

        if calls is None or puts is None:
            return None

        # Get current date and calculate days to expiry
        today = datetime.now()
        # Ensure expiry_date is a date object for comparison with today
        if isinstance(expiration_date, str):
            expiry_date = datetime.strptime(expiration_date, '%Y-%m-%d').date()
        else:
            expiry_date = expiration_date.date() # if it's already a datetime object

        # Calculate days to expiry. Add 1 to ensure options expiring today are considered 1 day out.
        days_to_expiry = (expiry_date - today.date()).days + 1
        if days_to_expiry <= 0:
            days_to_expiry = 0.001 # Smallest positive number to avoid division by zero

        # Prepare DataFrames: Add 'type' column
        calls['type'] = 'CALL'
        puts['type'] = 'PUT'

        # Combine and calculate DEX
        all_options = pd.concat([calls, puts])

        # Simplified delta calculation
        # For deep ITM calls: delta ~ 1, for deep OTM calls: delta ~ 0
        # For deep ITM puts: delta ~ -1, for deep OTM puts: delta ~ 0
        def calculate_delta(option_type, strike, price):
            # Using implied volatility from yfinance data for a more accurate delta calculation
            # If IV is not available, default to a simplified model
            S = self.underlying_price
            K = strike
            r = 0.02 # Risk-free rate
            T = days_to_expiry / 365.0

            # If impliedVolatility is available in the option chain, use it.
            # Otherwise, use a simplified approach or a default.
            # For now, let's stick to the simplified method as in the original code, but note the enhancement possibility.
            moneyness = S / K - 1
            if option_type == 'CALL':
                if moneyness > 0.05:  # ITM
                    return 0.7 + 0.3 * min(moneyness / 0.5, 1)
                elif moneyness < -0.05:  # OTM
                    return 0.3 * max(1 + moneyness / 0.5, 0)
                else:  # ATM
                    return 0.5
            else:  # PUT
                if moneyness < -0.05:  # ITM
                    return -0.7 - 0.3 * min(abs(moneyness) / 0.5, 1)
                elif moneyness > 0.05:  # OTM
                    return -0.3 * max(1 - moneyness / 0.5, 0)
                else:  # ATM
                    return -0.5

        all_options['delta'] = all_options.apply(
            lambda row: calculate_delta(
                'CALL' if row['type'] == 'CALL' else 'PUT',
                row['strike'],
                row['lastPrice']
            ), axis=1
        )

        # Calculate Delta Exposure (DEX)
        all_options['dex_oi'] = all_options['delta'] * all_options['openInterest'] * 100 * self.underlying_price
        all_options['dex_volume'] = all_options['delta'] * all_options['volume'] * 100 * self.underlying_price

        # Separate and aggregate
        calls = all_options[all_options['type'] == 'CALL'][['strike', 'dex_oi', 'dex_volume']]
        puts = all_options[all_options['type'] == 'PUT'][['strike', 'dex_oi', 'dex_volume']]

        merged = pd.merge(
            calls.rename(columns={'dex_oi': 'call_dex_oi', 'dex_volume': 'call_dex_volume'}),
            puts.rename(columns={'dex_oi': 'put_dex_oi', 'dex_volume': 'put_dex_volume'}),
            on='strike',
            how='outer'
        ).fillna(0)

        merged['net_dex_oi'] = merged['call_dex_oi'] + merged['put_dex_oi']  # Note: put deltas are negative
        merged['abs_dex_oi'] = abs(merged['call_dex_oi']) + abs(merged['put_dex_oi'])
        merged['net_dex_volume'] = merged['call_dex_volume'] + merged['put_dex_volume']
        merged['abs_dex_volume'] = abs(merged['call_dex_volume']) + abs(merged['put_dex_volume'])

        return merged

    def plot_all_metrics(self, expiration_date):
        """Create comprehensive plots of all GEX and DEX metrics"""
        # Get current price
        self.get_current_price()

        # Calculate DEX and GEX
        dex_df = self.calculate_dex(expiration_date)
        gex_df, all_options = self.calculate_gex(expiration_date)

        if dex_df is None or gex_df is None:
            print("Failed to fetch data")
            return

        # Merge DEX and GEX data
        merged_df = pd.merge(dex_df, gex_df, on='strike', how='outer').fillna(0)

        # Filter for relevant strikes (within 20% of current price)
        if self.underlying_price is None:
            print("Underlying price not available for filtering strikes.")
            filtered_df = merged_df
        else:
            price_range = 0.20
            min_strike = self.underlying_price * (1 - price_range)
            max_strike = self.underlying_price * (1 + price_range)
            filtered_df = merged_df[(merged_df['strike'] >= min_strike) & (merged_df['strike'] <= max_strike)]

        # Create Plotly figure with subplots (5 rows, 2 columns for 10 plots)
        fig = make_subplots(
            rows=5, cols=2,
            subplot_titles=(
                'DEX (Delta Exposure) - Open Interest',
                'Net GEX (Gamma Exposure) - Open Interest',
                'Absolute GEX - Open Interest',
                'Net GEX - Volume',
                'Absolute GEX - Volume',
                'Gamma by Strike (Raw Gamma)',
                'DEX Comparison: OI vs Volume',
                'GEX Comparison: OI vs Volume',
                'Total Gamma Profile (Net GEX)',
                'Total Gamma Profile (Absolute GEX)'
            )
        )

        # Plot 1: DEX (Delta Exposure) - Open Interest
        fig.add_trace(
            go.Bar(x=filtered_df['strike'], y=filtered_df['net_dex_oi'], name='Net DEX OI', marker_color='blue'),
            row=1, col=1
        )
        fig.add_hline(y=0, line_dash="dash", line_color="black", annotation_text="Zero DEX", row=1, col=1)
        if self.underlying_price:
            fig.add_vline(x=self.underlying_price, line_dash="dash", line_color="red", annotation_text=f'Spot: ${self.underlying_price:.2f}', row=1, col=1)

        # Plot 2: Net GEX (Gamma Exposure) - Open Interest
        fig.add_trace(
            go.Bar(x=filtered_df['strike'], y=filtered_df['net_gex_oi'], name='Net GEX OI', marker_color='green'),
            row=1, col=2
        )
        fig.add_hline(y=0, line_dash="dash", line_color="black", annotation_text="Zero GEX", row=1, col=2)
        if self.underlying_price:
            fig.add_vline(x=self.underlying_price, line_dash="dash", line_color="red", row=1, col=2)

        # Plot 3: Absolute GEX (Open Interest)
        fig.add_trace(
            go.Bar(x=filtered_df['strike'], y=filtered_df['abs_gex_oi'], name='Abs GEX OI', marker_color='orange'),
            row=2, col=1
        )
        if self.underlying_price:
            fig.add_vline(x=self.underlying_price, line_dash="dash", line_color="red", row=2, col=1)

        # Plot 4: Net GEX - Volume
        fig.add_trace(
            go.Bar(x=filtered_df['strike'], y=filtered_df['net_gex_volume'], name='Net GEX Volume', marker_color='purple'),
            row=2, col=2
        )
        fig.add_hline(y=0, line_dash="dash", line_color="black", annotation_text="Zero GEX", row=2, col=2)
        if self.underlying_price:
            fig.add_vline(x=self.underlying_price, line_dash="dash", line_color="red", row=2, col=2)

        # Plot 5: Absolute GEX - Volume
        fig.add_trace(
            go.Bar(x=filtered_df['strike'], y=filtered_df['abs_gex_volume'], name='Abs GEX Volume', marker_color='brown'),
            row=3, col=1
        )
        if self.underlying_price:
            fig.add_vline(x=self.underlying_price, line_dash="dash", line_color="red", row=3, col=1)

        # Plot 6: Gamma by strike (Calls vs Puts)
        if all_options is not None:
            if self.underlying_price:
                filtered_options = all_options[(all_options['strike'] >= min_strike) &
                                             (all_options['strike'] <= max_strike)]
            else:
                filtered_options = all_options

            calls_gamma = filtered_options[filtered_options['type'] == 'CALL']
            puts_gamma = filtered_options[filtered_options['type'] == 'PUT']

            fig.add_trace(
                go.Scatter(x=calls_gamma['strike'], y=calls_gamma['gamma'], mode='markers', name='Calls Gamma', marker=dict(color='green', size=8)),
                row=3, col=2
            )
            fig.add_trace(
                go.Scatter(x=puts_gamma['strike'], y=puts_gamma['gamma'], mode='markers', name='Puts Gamma', marker=dict(color='red', size=8)),
                row=3, col=2
            )
            if self.underlying_price:
                fig.add_vline(x=self.underlying_price, line_dash="dash", line_color="red", row=3, col=2)

        # Plot 7: Combined DEX view (OI + Volume)
        fig.add_trace(
            go.Bar(x=filtered_df['strike'], y=filtered_df['net_dex_oi'], name='DEX OI', marker_color='blue', opacity=0.7),
            row=4, col=1
        )
        fig.add_trace(
            go.Bar(x=filtered_df['strike'], y=filtered_df['net_dex_volume'], name='DEX Volume', marker_color='cyan', opacity=0.7),
            row=4, col=1
        )
        fig.add_hline(y=0, line_dash="dash", line_color="black", row=4, col=1)
        if self.underlying_price:
            fig.add_vline(x=self.underlying_price, line_dash="dash", line_color="red", row=4, col=1)

        # Plot 8: Combined GEX view (OI + Volume)
        fig.add_trace(
            go.Bar(x=filtered_df['strike'], y=filtered_df['net_gex_oi'], name='GEX OI', marker_color='green', opacity=0.7),
            row=4, col=2
        )
        fig.add_trace(
            go.Bar(x=filtered_df['strike'], y=filtered_df['net_gex_volume'], name='GEX Volume', marker_color='lightgreen', opacity=0.7),
            row=4, col=2
        )
        fig.add_hline(y=0, line_dash="dash", line_color="black", row=4, col=2)
        if self.underlying_price:
            fig.add_vline(x=self.underlying_price, line_dash="dash", line_color="red", row=4, col=2)

        # Plot 9: Total Gamma Profile (Net GEX)
        fig.add_trace(
            go.Scatter(x=filtered_df['strike'], y=filtered_df['net_gex_oi'], mode='lines', name='Net GEX OI', line=dict(color='green', width=2)),
            row=5, col=1
        )
        fig.add_trace(
            go.Scatter(x=filtered_df['strike'], y=filtered_df['net_gex_volume'], mode='lines', name='Net GEX Volume', line=dict(color='blue', dash='dash', width=2)),
            row=5, col=1
        )
        fig.add_hline(y=0, line_dash="dash", line_color="black", row=5, col=1)
        if self.underlying_price:
            fig.add_vline(x=self.underlying_price, line_dash="dash", line_color="red", row=5, col=1)

        # Plot 10: Total Gamma Profile (Absolute GEX)
        fig.add_trace(
            go.Scatter(x=filtered_df['strike'], y=filtered_df['abs_gex_oi'], mode='lines', name='Abs GEX OI', line=dict(color='orange', width=2)),
            row=5, col=2
        )
        fig.add_trace(
            go.Scatter(x=filtered_df['strike'], y=filtered_df['abs_gex_volume'], mode='lines', name='Abs GEX Volume', line=dict(color='brown', dash='dash', width=2)),
            row=5, col=2
        )
        if self.underlying_price:
            fig.add_vline(x=self.underlying_price, line_dash="dash", line_color="red", row=5, col=2)

        # Update layout for overall title and interactive features
        fig.update_layout(
            title_text=f'SPY Options Analysis - Expiry: {expiration_date}<br>Current Price: ${self.underlying_price:.2f}' if self.underlying_price else f'SPY Options Analysis - Expiry: {expiration_date}<br>Current Price: N/A',
            height=2000, width=1200, # Adjust overall figure size for 10 plots
            showlegend=True,
            hovermode='x unified',
            legend=dict(orientation="h", yanchor="bottom", y=-0.05, xanchor="center", x=0.5)
        )

        # Update x and y axes labels for all subplots
        for i in range(1, 6):
            for j in range(1, 3):
                fig.update_xaxes(title_text='Strike Price', row=i, col=j)
                # Specific y-axis labels based on the plot
                if (i, j) in [(1,1), (2,1), (4,1)]: # DEX related
                    fig.update_yaxes(title_text='Net DEX', row=i, col=j)
                elif (i, j) in [(1,2), (2,2), (3,1), (4,2), (5,1), (5,2)]: # GEX related (including new plot)
                    fig.update_yaxes(title_text='Gamma Exposure', row=i, col=j)
                elif (i, j) == (3,2): # Raw Gamma
                    fig.update_yaxes(title_text='Gamma', row=i, col=j)

        fig.show()

        # Print summary statistics
        print("\n" + "="*60)
        print("SUMMARY STATISTICS")
        print("="*60)
        print(f"Underlying Price: ${self.underlying_price:.2f}" if self.underlying_price else "Underlying Price: N/A")
        print(f"Expiration: {expiration_date}")
        if not filtered_df.empty:
            print(f"Total Net DEX (OI): ${filtered_df['net_dex_oi'].sum()/1e9:.3f}B")
            print(f"Total Net GEX (OI): ${filtered_df['net_gex_oi'].sum()/1e9:.3f}B")
            print(f"Total Abs GEX (OI): ${filtered_df['abs_gex_oi'].sum()/1e9:.3f}B")
            print(f"Total Net GEX (Volume): ${filtered_df['net_gex_volume'].sum()/1e9:.3f}B")
            print(f"Total Abs GEX (Volume): ${filtered_df['abs_gex_volume'].sum()/1e9:.3f}B")
        else:
            print("No filtered data available for summary statistics.")

        return filtered_df

    def plot_spy_with_greeks_levels(self):
        """Fetches 1-minute SPY data, calculates max abs GEX/DEX strikes for nearest expiry,
        and plots SPY with horizontal lines at these levels.
        """
        # 1. Fetch 1-minute historical data for SPY
        spy_1min_data = pd.DataFrame()
        try:
            spy_1min_data = self.ticker.history(period='1d', interval='1m')
            if spy_1min_data.empty:
                print("No 1-minute data available for today. Falling back to daily data for candlestick plot.")
                spy_1min_data = self.ticker.history(period='1d') # Fallback for candlestick
        except Exception as e:
            print(f"Error fetching 1-minute SPY data: {e}. Falling back to daily data for candlestick plot.")
            spy_1min_data = self.ticker.history(period='1d') # Fallback for candlestick

        # Ensure underlying_price is set for calculations (it will handle its own empty data)
        self.get_current_price()

        if self.underlying_price is None:
            print("Cannot plot Greek levels without a current underlying price.")
            return

        # 2. Retrieve available option expiration dates and select nearest expiry
        expirations = self.ticker.options
        if not expirations:
            print("No options data available.")
            return

        nearest_expiry = expirations[0]
        print(f"Analyzing nearest expiration for Greek levels: {nearest_expiry}")

        # 3. Calculate DEX and GEX for nearest expiry
        dex_df = self.calculate_dex(nearest_expiry)
        gex_df, _ = self.calculate_gex(nearest_expiry)

        if dex_df is None or gex_df is None:
            print("Failed to calculate DEX or GEX for the nearest expiration.")
            return

        # Filter for relevant strikes (within 20% of current price)
        price_range = 0.20
        min_strike = self.underlying_price * (1 - price_range)
        max_strike = self.underlying_price * (1 + price_range)

        # Merge DEX and GEX data for easier filtering and analysis
        merged_df = pd.merge(dex_df, gex_df, on='strike', how='outer').fillna(0)
        filtered_df = merged_df[(merged_df['strike'] >= min_strike) & (merged_df['strike'] <= max_strike)]

        # 4. Identify significant strike prices
        max_dex_strike = None
        max_gex_oi_strike = None
        max_gex_volume_strike = None

        max_pos_net_gex_oi_strike = None
        min_neg_net_gex_oi_strike = None
        max_pos_net_gex_volume_strike = None
        min_neg_net_gex_volume_strike = None

        if not filtered_df.empty:
            # Max Abs DEX OI
            if 'abs_dex_oi' in filtered_df.columns and not filtered_df['abs_dex_oi'].isnull().all():
                max_dex_strike = filtered_df.loc[filtered_df['abs_dex_oi'].idxmax()]['strike']

            # Max Abs GEX OI
            if 'abs_gex_oi' in filtered_df.columns and not filtered_df['abs_gex_oi'].isnull().all():
                max_gex_oi_strike = filtered_df.loc[filtered_df['abs_gex_oi'].idxmax()]['strike']

            # Max Abs GEX Volume
            if 'abs_gex_volume' in filtered_df.columns and not filtered_df['abs_gex_volume'].isnull().all():
                max_gex_volume_strike = filtered_df.loc[filtered_df['abs_gex_volume'].idxmax()]['strike']

            # Net GEX OI (positive and negative peaks)
            positive_net_gex_oi = filtered_df[filtered_df['net_gex_oi'] > 0]
            if not positive_net_gex_oi.empty and not positive_net_gex_oi['net_gex_oi'].isnull().all():
                max_pos_net_gex_oi_strike = positive_net_gex_oi.loc[positive_net_gex_oi['net_gex_oi'].idxmax()]['strike']

            negative_net_gex_oi = filtered_df[filtered_df['net_gex_oi'] < 0]
            if not negative_net_gex_oi.empty and not negative_net_gex_oi['net_gex_oi'].isnull().all():
                min_neg_net_gex_oi_strike = negative_net_gex_oi.loc[negative_net_gex_oi['net_gex_oi'].idxmin()]['strike']

            # Net GEX Volume (positive and negative peaks)
            positive_net_gex_volume = filtered_df[filtered_df['net_gex_volume'] > 0]
            if not positive_net_gex_volume.empty and not positive_net_gex_volume['net_gex_volume'].isnull().all():
                max_pos_net_gex_volume_strike = positive_net_gex_volume.loc[positive_net_gex_volume['net_gex_volume'].idxmax()]['strike']

            negative_net_gex_volume = filtered_df[filtered_df['net_gex_volume'] < 0]
            if not negative_net_gex_volume.empty and not negative_net_gex_volume['net_gex_volume'].isnull().all():
                min_neg_net_gex_volume_strike = negative_net_gex_volume.loc[negative_net_gex_volume['net_gex_volume'].idxmin()]['strike']


        print(f"Current SPY Price: {self.underlying_price:.2f}")
        if max_dex_strike: print(f"Strike with Max Abs DEX OI: {max_dex_strike:.2f}")
        if max_gex_oi_strike: print(f"Strike with Max Abs GEX OI: {max_gex_oi_strike:.2f}")
        if max_gex_volume_strike: print(f"Strike with Max Abs GEX Volume: {max_gex_volume_strike:.2f}")
        if max_pos_net_gex_oi_strike: print(f"Strike with Max Pos Net GEX OI: {max_pos_net_gex_oi_strike:.2f}")
        if min_neg_net_gex_oi_strike: print(f"Strike with Max Neg Net GEX OI: {min_neg_net_gex_oi_strike:.2f}")
        if max_pos_net_gex_volume_strike: print(f"Strike with Max Pos Net GEX Volume: {max_pos_net_gex_volume_strike:.2f}")
        if min_neg_net_gex_volume_strike: print(f"Strike with Max Neg Net GEX Volume: {min_neg_net_gex_volume_strike:.2f}")


        # 5. Create Plotly figure
        fig = go.Figure()

        # 6. Add candlestick trace if 1-minute data is available and meaningful
        if not spy_1min_data.empty and 'Open' in spy_1min_data.columns:
            fig.add_trace(go.Candlestick(
                x=spy_1min_data.index,
                open=spy_1min_data['Open'],
                high=spy_1min_data['High'],
                low=spy_1min_data['Low'],
                close=spy_1min_data['Close'],
                name='SPY Candlestick',
                increasing_line_color='green',
                decreasing_line_color='red'
            ))
        else:
            print("1-minute SPY data is not suitable for candlestick plot.")
            # If no 1-min data, plot current price as a dot
            if self.underlying_price:
                 fig.add_trace(go.Scatter(
                    x=[datetime.now()],
                    y=[self.underlying_price],
                    mode='markers',
                    marker=dict(size=10, color='blue'),
                    name='Current SPY Price' # Legend will show for this if candlestick is not plotted.
                 ))

        # 7. Add horizontal lines for significant Greek levels
        if max_dex_strike:
            fig.add_hline(y=max_dex_strike, line_dash="dash", line_color="blue",
                          annotation_text=f'Max Abs DEX OI: {max_dex_strike:.2f}',
                          annotation_position="bottom right",
                          annotation_font_color="blue")
        if max_gex_oi_strike:
            fig.add_hline(y=max_gex_oi_strike, line_dash="dot", line_color="green",
                          annotation_text=f'Max Abs GEX OI: {max_gex_oi_strike:.2f}',
                          annotation_position="top right",
                          annotation_font_color="green")
        if max_gex_volume_strike:
            fig.add_hline(y=max_gex_volume_strike, line_dash="dot", line_color="darkgreen",
                          annotation_text=f'Max Abs GEX Vol: {max_gex_volume_strike:.2f}',
                          annotation_position="bottom left",
                          annotation_font_color="darkgreen")

        if max_pos_net_gex_oi_strike:
            fig.add_hline(y=max_pos_net_gex_oi_strike, line_dash="dashdot", line_color="orange",
                          annotation_text=f'Max Pos Net GEX OI: {max_pos_net_gex_oi_strike:.2f}',
                          annotation_position="top left",
                          annotation_font_color="orange")
        if min_neg_net_gex_oi_strike:
            fig.add_hline(y=min_neg_net_gex_oi_strike, line_dash="dashdot", line_color="purple",
                          annotation_text=f'Max Neg Net GEX OI: {min_neg_net_gex_oi_strike:.2f}',
                          annotation_position="bottom left",
                          annotation_font_color="purple")
        if max_pos_net_gex_volume_strike:
            fig.add_hline(y=max_pos_net_gex_volume_strike, line_dash="longdash", line_color="brown",
                          annotation_text=f'Max Pos Net GEX Vol: {max_pos_net_gex_volume_strike:.2f}',
                          annotation_position="top right",
                          annotation_font_color="brown")
        if min_neg_net_gex_volume_strike:
            fig.add_hline(y=min_neg_net_gex_volume_strike, line_dash="longdash", line_color="pink",
                          annotation_text=f'Max Neg Net GEX Vol: {min_neg_net_gex_volume_strike:.2f}',
                          annotation_position="bottom right",
                          annotation_font_color="pink")

        # 8. Add a horizontal line for the current self.underlying_price
        if self.underlying_price:
            fig.add_hline(y=self.underlying_price, line_dash="solid", line_color="red",
                          annotation_text=f'Current SPY Price: {self.underlying_price:.2f}',
                          annotation_position="top left",
                          annotation_font_color="red")

        # 9. Set appropriate title and labels
        fig.update_layout(
            title_text=f'SPY 1-Minute Chart with Greek Levels (Expiry: {nearest_expiry})',
            xaxis_title='Time',
            yaxis_title='Price',
            xaxis_rangeslider_visible=False, # Hide range slider for better view
            height=700, width=1000,
            showlegend=False # Legends for lines are handled by annotations
        )

        # 10. Display the plot
        fig.show()

# Usage
def main():
    # Initialize analyzer
    analyzer = GammaExposureAnalyzer("SPY")

    # Get available expirations
    # The line below is redundant as it's called inside the new method anyway
    # analyzer.ticker = yf.Ticker("^SPY")
    expirations = analyzer.ticker.options

    if not expirations:
        print("No options data available for SPY")
        return

    print(f"Available expirations: {expirations[:5]}")

    # Get current price
    current_price = analyzer.get_current_price()
    if current_price is None:
        print("Could not retrieve current SPY price. Exiting.")
        return
    print(f"Current SPY Price: ${current_price:.2f}")

    # Use nearest expiration for plot_all_metrics
    nearest_expiry = expirations[0]
    print(f"\nAnalyzing nearest expiration for full metrics: {nearest_expiry}")

    # Plot all metrics
    _ = analyzer.plot_all_metrics(nearest_expiry)

    print("\n" + "="*60)
    print("GENERATING SPY CHART WITH GREEK LEVELS")
    print("="*60)
    analyzer.plot_spy_with_greeks_levels()

if __name__ == "__main__":
    main()

Available expirations: ('2026-01-13', '2026-01-14', '2026-01-15', '2026-01-16', '2026-01-20')
Current SPY Price: $694.85

Analyzing nearest expiration for full metrics: 2026-01-13



SUMMARY STATISTICS
Underlying Price: $694.85
Expiration: 2026-01-13
Total Net DEX (OI): $0.000B
Total Net GEX (OI): $0.000B
Total Abs GEX (OI): $0.000B
Total Net GEX (Volume): $23.246B
Total Abs GEX (Volume): $43.883B

GENERATING SPY CHART WITH GREEK LEVELS
Analyzing nearest expiration for Greek levels: 2026-01-13
Current SPY Price: 694.85
Strike with Max Abs DEX OI: 655.00
Strike with Max Abs GEX OI: 560.00
Strike with Max Abs GEX Volume: 695.00
Strike with Max Pos Net GEX Volume: 695.00
Strike with Max Neg Net GEX Volume: 694.00
